# Setup and Imports

In [ ]:
%pip install qutip --upgrade
%pip install matplotlib --upgrade
# for progress bar
%pip install tqdm --upgrade
# for latex from function to use in plots
%pip install latexify-py --upgrade
# parallel process
%pip install loky --upgrade
# for running julia scripts inside python
#%pip install julia --upgrade

In [ ]:
import matplotlib.pyplot as plt # type: ignore
import numpy as np # type: ignore
import qutip as qt # type: ignore
from tqdm.auto import tqdm # type: ignore
from scipy.optimize import curve_fit # type: ignore
import latexify # type: ignore
from functools import partial
from IPython.display import display

b_odmr=500.0
run_odmr=False
save_odmr=False
run_contrast=False
save_contrast=False
run_rabi=False
E=0.0
save_rabi=False


General functions to be used in the simulations

In [ ]:
def norm(arr):
    """
    Normalize an array by scaling its values between 0 and 1.

    Parameters:
    arr (numpy.ndarray): The input array.

    Returns:
    numpy.ndarray: The normalized array.
    """
    return (arr-np.min(arr))/(np.max(arr)-np.min(arr))

def stand(arr):
    """
    Standardizes an array by subtracting the mean and dividing by the standard deviation.

    Parameters:
    arr (array-like): The input array to be standardized.

    Returns:
    array-like: The standardized array.

    """
    return (arr-np.mean(arr))/np.std(arr)
def max_reg(arr):
    """
    Normalize the input array by dividing it by its maximum value.

    Parameters:
    arr (numpy.ndarray): The input array to be normalized.

    Returns:
    numpy.ndarray: The normalized array.

    """
    return arr/np.max(arr)

def B0(b_0, phi, theta):
    """
    Calculate the magnetic field vector in Cartesian coordinates.
    Parameters:
    - b_0 (float): The magnitude of the magnetic field.
    - theta (float): The azimuthal angle in radians.
    - phi (float): The polar angle in radians.
    Returns:
    - list: The magnetic field vector in Cartesian coordinates [Bx, By, Bz].
    """
    return np.array([b_0 * np.sin(theta) * np.cos(phi), b_0 * np.sin(theta) * np.sin(phi), b_0 * np.cos(theta)])

def construct_spin_matrices(basis):
    """
    Constructs the spin matrices for a given basis.
    Parameters:
    - basis (list): A list of basis elements.
    Returns:
    - s_x (array): The x-component of the spin matrix.
    - s_y (array): The y-component of the spin matrix.
    - s_z (array): The z-component of the spin matrix.
    Raises:
    - ValueError: If the number of basis elements is not 3.
    """
    if len(basis) == 3:
        # 3-level system (spin-1)
        s_x = (basis[0] * basis[1].dag() + basis[1] * basis[0].dag() +
               basis[1] * basis[2].dag() + basis[2] * basis[1].dag()) / np.sqrt(2)
        
        s_y = -1j * (basis[0] * basis[1].dag() - basis[1] * basis[0].dag() +
                     basis[1] * basis[2].dag() - basis[2] * basis[1].dag()) / np.sqrt(2)
        
        s_z = basis[0] * basis[0].dag() - basis[2] * basis[2].dag()
        return s_x, s_y, s_z

    else:
        raise ValueError("This function supports only 3 basis elements.")

# Original Model by Magaletti et al.

## Introduction
This model explores the control of NV centers using continuous lasers as described by Magaletti et al.

## Mathematical Formulation

The Hamiltonian for the system is given by
$$
H_{mg}=\frac{\Omega_r}{2} \left( \ket{0}\bra{-1}e^{i \omega t} + \ket{-1}\bra{0}e^{-i \omega t}\right)
$$

laser driving terms
$$
L_{1 4} = \sqrt{W_p} \ket{4}\bra{1} \\
L_{2 5} = \sqrt{W_p} \ket{5}\bra{2} \\
L_{3 6} = \sqrt{W_p} \ket{6}\bra{3}
$$
Optical transition terms
$$
L_{4 1} = \sqrt{k_{41}} \ket{1}\bra{4}\\
L_{5 2} = \sqrt{k_{52}} \ket{2}\bra{5}\\
L_{6 3} = \sqrt{k_{63}} \ket{3}\bra{6}\\
L_{4 7} = \sqrt{k_{47}} \ket{7}\bra{4}\\
L_{5 7} = \sqrt{k_{57}} \ket{7}\bra{5}\\
L_{6 7} = \sqrt{k_{67}} \ket{7}\bra{6}\\
L_{7 1} = \sqrt{k_{71}} \ket{1}\bra{7}\\
L_{7 2} = \sqrt{k_{72}} \ket{2}\bra{7}\\
L_{7 3} = \sqrt{k_{73}} \ket{3}\bra{7}
$$
Relaxation and dephasing terms
$$
L_{T_{1,j}}= \sqrt{\gamma^{j}_1/2} \ \sigma^{j}_-\\
L_{T^*_{2,j}}= \sqrt{\gamma^{j}_2} \ \sigma^{j}_z\\
j=\{gs,es\}, \ \gamma^{j}_1 = 1/T_{1,j}, \ \gamma^{j}_2 = 1/T^*_{2,j} \\
T_{1,gs} \approx 10^3 \ \mu s , \ T^*_{2,gs} \approx 2 \ \mu s  \\
T_{1,es} \approx 10^3 \ \mu s  , \ T^*_{2,es} \approx 6 \cdot 10^{-3} \ \mu s 
$$


## Simulation Code

### Hamiltonian and constants

In [ ]:
# Dimention of the Hilbert space
dim = 7
k_ind=2
# dephasing times(microseconds)
t1_gs=1e3
t2_gs=2
t1_es=1e3
t2_es=6*1e-3
# Constants (MHz)
gamma_gs = 1/t1_gs,1/t2_gs
gamma_es = 1/t1_es,1/t2_es
W_p = 1.9
# I had to modify this two value to repruduce the results of the paper
Om_r = 15.7
omega = 0.1

# States (I broke them in a few lines to be more readable)
excited = qt.basis(dim, 4), qt.basis(dim, 3),qt.basis(dim, 5) # |+1>_es, |0>_es, |-1>_es
isc = qt.basis(dim, 6)
ground = qt.basis(dim, 1), qt.basis(dim, 0), qt.basis(dim, 2) # |+1>_gs, |0>_gs, |-1>_gs
IdNV=qt.qeye(dim)

n1, n2, n3 = (
    ground[1] * ground[1].dag(), #type: ignore
    ground[2] * ground[2].dag(), #type: ignore
    ground[0] * ground[0].dag(), #type: ignore
)
n4, n5, n6 = (
    excited[1] * excited[1].dag(), #type: ignore
    excited[2] * excited[2].dag(), #type: ignore
    excited[0] * excited[0].dag(), #type: ignore
)
n7, nc = (
    isc * isc.dag(), #type: ignore
    ground[2] * ground[1].dag(), #type: ignore
)

sx_gs, sy_gs, sz_gs = construct_spin_matrices(ground)
sx_es, sy_es, sz_es = construct_spin_matrices(excited)
sm_gs,sp_gs= sx_gs - 1j * sy_gs, sx_gs + 1j * sy_gs
sm_es,sp_es= sx_es - 1j * sy_es, sx_es + 1j * sy_es

S_gs=np.array([sx_gs, sy_gs, sz_gs])
S_es=np.array([sx_es, sy_es, sz_es])

def H_mg(om_r):
    """Returns the Hamiltonian of the system based on whether the MW is on or off
    Parameters:
        om_r (float) - Rabi frequency

    Returns:
        H_0 (list) - list of the Hamiltonian terms and their time dependence
    """
    H_0 = [[0.5 * om_r * (ground[1]*ground[2].dag()), "exp(1j*w*t)"],  #type: ignore
           [0.5 * om_r * (ground[2]*ground[1].dag()), "exp(-1j*w*t)"]] #type: ignore
    return H_0


def L_mg(w_p,k_index=k_ind):
    """Returns the Lindblad operators of the system.
    
    Parameters:
        w_p (float) - Laser pump rate
        
    Returns:
        c_ops (list) - list of the Lindblad operators
    """
    K_s=[[66.0,0.0,57.0,1.0,0.7],
         [77.0,0.0,30.0,3.3,0.0],
         [62.7,12.97,80.0,3.45,1.08],
         [63.2,10.8,60.7,0.8,0.4],
         [67.4,9.9,96.6,4.83,1.055]]
    k41 = K_s[k_index][0]
    k52 = K_s[k_index][0]
    k63 = K_s[k_index][0]
    k57 = K_s[k_index][2]
    k67 = K_s[k_index][2]
    k47 = K_s[k_index][1]
    k71 = K_s[k_index][3]
    k72 = K_s[k_index][4]
    k73 = K_s[k_index][4]
    
    c_ops = []

    c_ops.append(np.sqrt(w_p) * (excited[1] * ground[1].dag()))  # n1 to n4 #type: ignore 
    c_ops.append(np.sqrt(w_p) * (excited[2] * ground[2].dag()))  # n2 to n5 #type: ignore
    c_ops.append(np.sqrt(w_p) * (excited[0] * ground[0].dag())) # n3 to n6  #type: ignore

    c_ops.append(np.sqrt(k41) * (ground[1] * excited[1].dag()))  # n4 to n1 #type: ignore
    c_ops.append(np.sqrt(k71) * (ground[1] * isc.dag()))  # n7 to n1    #type: ignore

    c_ops.append(np.sqrt(k52) * (ground[2] * excited[2].dag()))  # n5 to n2 #type: ignore
    c_ops.append(np.sqrt(k72) * (ground[2] * isc.dag()))  # n7 to n2 #type: ignore

    c_ops.append(np.sqrt(k63) * (ground[0] * excited[0].dag()))  # n6 to n3 #type: ignore
    c_ops.append(np.sqrt(k73) * (ground[0] * isc.dag()))  # n7 to n3    #type: ignore

    c_ops.append(np.sqrt(k47) * (isc * excited[1].dag()))  # n4 to n7   #type: ignore
    c_ops.append(np.sqrt(k57) * (isc * excited[2].dag()))  # n5 to n7   #type: ignore
    c_ops.append(np.sqrt(k67) * (isc * excited[0].dag()))  # n6 to n7   #type: ignore
    
    # Add collapse operators for decoherence
    c_ops.append(np.sqrt(gamma_gs[1]-gamma_gs[0]/2) * sz_gs)
    c_ops.append(np.sqrt(gamma_gs[0]/2) * (sm_gs))
    c_ops.append(np.sqrt(gamma_gs[0]/2) * (sp_gs))
    c_ops.append(np.sqrt(gamma_es[1]) * sz_es)
    c_ops.append(np.sqrt(gamma_es[0]/2) * (sm_es))
    c_ops.append(np.sqrt(gamma_es[0]/2) * (sp_es))
    return c_ops

def dynamics_mg(
    dt,
    init_state,
    om=None,
    om_r=None,
    w_p=None,
    k_index=k_ind,
    exp_ops=None,
    ti=0.0,    
    mode="Free",
    progress_bar="ON",
    i=0,
):
    """
    Perform dynamics_mg simulation based on the given parameters, including optical transition rates index.
    Where, when using k_index=2 -> K_s[k_index]=[62.7,12.97,80.0,3.45,1.08], and the full K_s list is:
        K_s=[[66.0,0.0,57.0,1.0,0.7],
             [77.0,0.0,30.0,3.3,0.0],
             [62.7,12.97,80.0,3.45,1.08],
             [63.2,10.8,60.7,0.8,0.4],
             [67.4,9.9,96.6,4.83,1.055]]
    Parameters:
    - dt (float): Time step for the calculations.
    - init_state: Initial state for the simulation.
    - om (float, optional): Angular frequency of the system. Defaults to omega.
    - om_r (float, optional): Angular frequency for MW-ON evolution. Defaults to Om_r.
    - w_p (float, optional): Frequency for laser-ON evolution. Defaults to W_p.
    - k_index=k_index (int, optional): Index for the optical transition rates. Defaults to k.
    - exp_ops (list, optional): List of expectation operators. Defaults to 
        [n1, n2, n3, n4, n5, n6, n7, nc, 
        sx_gs, sy_gs, sz_gs, 
        sx_es, sy_es, sz_es].
    - ti (float, optional): Initial time for the simulation. Defaults to 0.0.
    - mode (str, optional): Mode of the simulation. Can be "Free", "MW", "Laser", or "Laser-MW". Defaults to "Free".
    - progress_bar (str, optional): Progress bar option. Can be "ON" or "OFF". Defaults to "ON".
    - i (int, optional): Iteration number. Defaults to 0.

    Returns:
    - tf (float): Final time of the simulation.
    - result: Result of the simulation.
    """
    # Default values
    if om is None: om = omega
    if om_r is None: om_r = Om_r
    if w_p is None: w_p = W_p
    if exp_ops is None: exp_ops = [
        n1, n2, n3, n4, n5, n6, n7, nc, 
        sx_gs, sy_gs, sz_gs, 
        sx_es, sy_es, sz_es
    ]

    # Arguments for the Hamiltonian
    args = {"w": om}
    
    # Define the time resolution
    t_bins = 1000 if dt <= 5 else 5000
    
    tf = ti + dt

    # Define collapse operators and Hamiltonian based on mode
    match mode:
        case "Free":
            c_ops = L_mg(0.0, k_index=k_index)
            H = H_mg(0.0)
        case "MW":
            c_ops = L_mg(0.0, k_index=k_index)
            H = H_mg(om_r)
        case "Laser":
            c_ops = L_mg(w_p, k_index=k_index)
            H = H_mg(0.0)
        case "Laser-MW":
            c_ops = L_mg(w_p, k_index=k_index)
            H = H_mg(om_r)
        case _:
            raise ValueError('mode must be one of "Free", "MW", "Laser", or "Laser-MW"')
    
    # Call the master equation solver
    match progress_bar:
        case "OFF":
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                e_ops=exp_ops,
                args=args,
                options={"store_states": True},
            )
        case "ON":
            print(f"{mode} {int(i + 1)} \n ti | tf \n {int(ti)} | {int(tf)}")
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                e_ops=exp_ops,
                args=args,
                options={"store_states": True, "progress_bar": "tqdm"},
            )
        case _:
            raise ValueError('progress_bar must be "ON" or "OFF"')
    
    return tf, result

### Plot functions

In [ ]:
import matplotlib as mpl

default_area=6*4
new_figsize=(14,8)
new_area=new_figsize[0]*new_figsize[1]
scale=new_area/default_area
mpl.rcParams['lines.linewidth']=0.6*scale
mpl.rc('xtick', labelsize=16)           # x-tick numbers only
mpl.rc('ytick', labelsize=16)           # y-tick numbers only

In [ ]:
def plot_continuous(n_exp,times,tis,tfs):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "gold",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    for i in range(len(n_exp) - 1):
        plt.plot(times, n_exp[i], label=f"$n_{i+1}$", color=colors[i])
    plt.plot(times, n_exp[-1], label="$n_c$", color=colors[-1])

    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    n = 0
    eps = 5*1e-2
    for i in zip(tis, tfs):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(n_exp) - eps, np.max(n_exp) + eps],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.75 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(n_exp) - eps, np.max(n_exp) + eps],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.75 ** (int(n / 3) + 1),
                    label=f"MS depl #{int(n/3)+1}",
                )
            else:
                plt.fill_betweenx(
                    [np.min(n_exp) - eps, np.max(n_exp) + eps],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.75 ** (int(n / 3) + 1),
                    label=f"MW ON #{int(n/3)+1}",
                )
        n += 1
    plt.ylabel(f"Population",fontsize=20)
    plt.xlabel(r"Time($\mu$s)",fontsize=20)

    # set the time limits you want to show in the plot
    t_lim = (-5 * eps, tfs[-1] + 5 * eps)
    plt.xlim(t_lim)
    if tfs[-1]<30:
        plt.xticks(np.arange(0,tfs[-1]+1,1))
    plt.minorticks_on()
    plt.ylim((np.min(n_exp) - eps, np.max(n_exp) + eps))
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.show()

In [ ]:
def plot_continuous_5_rep_v_1(n_exp,times,tis,tfs,n_reps):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "gold",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    # Defines figure size
    fig,axs=plt.subplots(2,1,figsize=(10, 8),gridspec_kw={'height_ratios': [1, 2]})
    # Plot the population of each state
    for i in range(len(n_exp) - 1):
        axs[0].plot(times, n_exp[i], color=colors[i])
    axs[0].plot(times, n_exp[-1], color=colors[-1])

    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    eps = 5*1e-2
    for ax in axs:
        n = 0
        for i in zip(tis, tfs):
            if n % 3 == 0:
                ax.fill_betweenx(
                    [np.min(n_exp) - eps, np.max(n_exp) + eps],
                    i[0],
                    i[1],
                    color="palegreen",
                    alpha=0.3+0.55 ** (int(n / 3) + 1),
                    label=f"Laser ON" if ax==axs[0] and n<3 else "",
                )
            else:
                if (n - 1) % 3 == 0:
                    ax.fill_betweenx(
                        [np.min(n_exp) - eps, np.max(n_exp) + eps],
                        i[0],
                        i[1],
                        color="lightblue",
                        alpha=0.3+0.55 ** (int(n / 3) + 1),
                        label=f"MS depl" if ax==axs[0] and n<3 else "",
                    )
                else:
                    ax.fill_betweenx(
                        [np.min(n_exp) - eps, np.max(n_exp) + eps],
                        i[0],
                        i[1],
                        color="orchid",
                        alpha=0.3+0.55 ** (int(n / 3) + 1),
                        label=f"MW ON" if ax==axs[0] and n<3 else "",
                    )
            n += 1
    fig.text(0.02,0.5,f"Population",va='center',rotation='vertical',fontsize=20)
    axs[1].set_xlabel(r"Time($\mu$s)",fontsize=20)

    # set the time limits you want to show in the plot
    t_lim = (-0.5-5 * eps, tfs[-1] + 5 * eps+1)
    axs[0].set_xlim(t_lim)
    for ax in axs:
        ax.minorticks_on()
        ax.set_ylim((np.min(n_exp) - eps, np.max(n_exp) + eps))
    n_exp_1,times_1=n_exp[:,:len(times)//n_reps+1000], times[:len(times)//n_reps+1000]
    t_lim = (-5 * eps, times_1[-1] + 5 * eps)
    axs[1].set_xticks(np.arange(0,t_lim[1],1))
    axs[1].set_xlim(t_lim)
    for i in range(len(n_exp) - 1):
        axs[1].plot(times_1, n_exp_1[i], label=f"$n_{i+1}$", color=colors[i], lw=3)
    axs[1].plot(times_1, n_exp_1[-1],label=f"$n_c$", color=colors[-1], lw=3)
    axs[1].legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    axs[0].legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.show()

## Results and Discussion

In [ ]:
# Initial state where the ground state is evenly populated
init_state = 1 / 3 * (n1 + n2 + n3)

# Array to collect the result Class
results = np.array([])

# Initial time of the evolution
ti = 0.0

# Store the times of activation and deativation of laser and microwaves for region highlight in plotting
tis_mg = []
tfs_mg = []

# Set the number of times to repeat the protocol
n_reps = 5
# Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
dt_laser = 10.0
dt_free = 1.0
dt_mw = 1.0
print("Starting calculations, this may take a while...")
# This loop is defined to reproduce the protocol of Magalleti el al
for i in tqdm(range(n_reps)):
    tis_mg.append(ti)
    # Here Laser is ON
    # Run the dynamics based on the mode choosen
    tf, result = dynamics_mg(
        dt_laser,
        init_state,
        ti=ti,
        mode="Laser",
        i=i
    ) # type: ignore
    # Store the Result class for when the Laser is on
    results = np.append(results, result) # type: ignore
    # Save the initial state for the next part of the protocol
    init_state = result.states[-1]
    # Make the initial time the end time of the previous part of the procotol
    ti = tf
    tfs_mg.append(tf)
    # Run the dynamics based on the mode choosen
    tf, result = dynamics_mg(
        dt_free,
        init_state,
        ti=ti,
        i=i
    ) # type: ignore
    # Store the Result class for MW on evolution
    results = np.append(results, result) # type: ignore
    # Save the times for coloring areas of the plot
    tis_mg.append(ti)
    tfs_mg.append(tf)
    # Make initial time to be the end time of the last part of the protocol
    ti = tf
    # Make the last state  of this evolution step be the initial states of the next iteration
    init_state = result.states[-1]
    # Run the dynamics_mg based on the mode choosen
    tf, result = dynamics_mg(dt_mw, 
                             init_state, 
                             ti=ti, 
                             mode="MW", 
                             i=i) # type: ignore
    # Store the Result class for MW on evolution
    results = np.append(results, result) # type: ignore
    # Save the times for coloring areas of the plot
    tis_mg.append(ti)
    tfs_mg.append(tf)
    # Make initial time to be the end time of the last part of the protocol
    ti = tf
    # Make the last state  of this evolution step be the initial states of the next iteration
    init_state = result.states[-1]
# Gather the expectation values from the results array
for j in range(len(results[0].expect)):
    nn_exp = np.array([])
    for i in range(len(results)):
        nn_exp = np.append(nn_exp, results[i].expect[j])
    if j == 0:
        n_exp_mg = np.array([nn_exp])
    else:
        n_exp_mg = np.append(n_exp_mg, [nn_exp], axis=0)
# Gather the times for the plots
times_mg = np.array([])
for i in range(len(results)):
    times_mg = np.append(times_mg, results[i].times)
if len(n_exp_mg)>8:
    s_exp_mg=n_exp_mg[len(n_exp_mg)-6:]
    n_exp_mg=n_exp_mg[:len(n_exp_mg)-6]
print("Calculations are finished!! Go to the next block for plots")

Plots

In [ ]:
plot_continuous(n_exp_mg, times_mg, tis_mg, tfs_mg)

In [ ]:
if n_reps>1:
    plot_continuous_5_rep_v_1(n_exp_mg, times_mg, tis_mg, tfs_mg,n_reps)

In [ ]:
c = 0
print("Starting calculations, this may take a while...")
for dt_mw in tqdm([0.0,0.5*np.pi/Om_r,np.pi/Om_r],position=0):# Initial state where the ground state is evenly populated
    init_state = n1 
    if dt_mw==0.5*np.pi/Om_r:
        init_state = (n1 + n2)/2
    if dt_mw==np.pi/Om_r:   
        init_state = n2
    
    # Array to collect the result Class
    results_t = np.array([])
    
    # Initial time of the evolution
    ti = 0.0
    
    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis_t = []
    tfs_t = []
    
    # Set the number of times to repeat the protocol
    n = 1
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = 10.0
    dt_free = 1.0
    # This loop is defined to reproduce the protocol of Magalleti el al
    for i in tqdm(range(n),position=1,leave=False):
        tis_t.append(ti)
        # Here Laser is ON
        # Run the dynamics_no based on the mode choosen
        tf, result = dynamics_mg(
            dt_laser,
            init_state,
            ti=ti,
            mode="Laser",
            progress_bar="ON",
            i=i,
        ) # type:ignore
        # Store the Result class for when the Laser is on
        results_t = np.append(results_t, result) # type: ignore 
        # Save the initial state for the next part of the protocol
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs_t.append(tf)
    # Gather the expectation values from the results array
    for j in range(len(results_t[0].expect)):
        nn_exp = np.array([])
        for i in range(len(results_t)):
            nn_exp = np.append(nn_exp, results_t[i].expect[j])
        if j == 0:
            n_exp_t = np.array([nn_exp])
        else:
            n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
    # Gather the times for the plots
    times_t = np.array([])
    for i in range(len(results_t)):
        times_t = np.append(times_t, results_t[i].times)
    pl=np.array([])
    pl=np.add(n_exp_t[3],n_exp_t[4])
    pl=np.add(pl,n_exp_t[5])
    if c==0:
        pls_pure = np.array([pl])
        c+=1
    else:
        pls_pure = np.append(pls_pure,[pl],axis=0)
print(f"Calculations are finished!! See the plots below")
# Set the colors of each state
colors = [
    "dodgerblue",
    "chocolate",
    "gold",
    "mediumpurple",
    "mediumseagreen",
    "lightskyblue",
    "magenta",
    "mediumvioletred",
    "forestgreen",
]
# Defines figure size
plt.figure(figsize=(14, 8))
a=0
state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
for pl in pls_pure:
    plt.plot(times_t[0:],pl[0:], color=colors[a], label=f"init state=${state[a]}$")
    a+=1
times_t_pure=times_t[0:].copy()
eps_y = abs(np.min(pls_pure) - np.max(pls_pure))*5e-2
eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
plt.ylabel(f"PL(a.u.)",fontsize=20)
plt.xlabel(r"Time($\mu$s)",fontsize=20)

# set the time limits you want to show in the plot
t_lim = (times_t[0]-eps_x, times_t[-1] + eps_x)
plt.xlim(t_lim)
plt.ylim((np.min(pls_pure) - eps_y, np.max(pls_pure) + eps_y))
plt.minorticks_on()

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.show()

In [ ]:
c = 0
print("Starting calculations, this may take a while...")
for dt_mw in tqdm([0.0,0.5*np.pi/Om_r,np.pi/Om_r],position=0):# Initial state where the ground state is evenly populated
    init_state = 1 / 3 * (n1 + n2 + n3)
    
    # Array to collect the result Class
    results_t = np.array([])
    
    # Initial time of the evolution
    ti = 0.0
    
    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis_t = []
    tfs_t = []
    
    # Set the number of times to repeat the protocol
    n = 1
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = 10.0
    dt_free = 1.0
    # This loop is defined to reproduce the protocol of Magalleti el al
    for i in tqdm(range(n),position=1,leave=False):
        tis_t.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg(
            dt_laser,
            init_state,
            ti=ti,
            mode="Laser",
            progress_bar="ON",
            i=i,
        ) # type:ignore
        # Store the Result class for when the Laser is on
        results_t = np.append(results_t, result) # type: ignore 
        # Save the initial state for the next part of the protocol
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs_t.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg(
            dt_free,
            init_state,
            ti=ti,
            mode="Free",
            progress_bar="ON",
            i=i,
        ) # type:ignore
        # Store the Result class for MW on evolution
        results_t = np.append(results_t, result) # type: ignore 
        # Save the times for coloring areas of the plot
        tis_t.append(ti)
        tfs_t.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg(dt_mw, 
                              init_state, 
                              ti=ti, 
                              mode="MW", 
                              progress_bar="ON", 
                              i=i) # type:ignore
        # Store the Result class for MW on evolution
        results_t = np.append(results_t, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis_t.append(ti)
        tfs_t.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
        tis_t.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg(
            dt_laser,
            init_state,
            ti=ti,
            mode="Laser",
            progress_bar="ON",
            i=i+1,
        ) # type:ignore
        # Store the Result class for when the Laser is on
        results_t = np.append(results_t, result) # type: ignore 
        # Save the initial state for the next part of the protocol
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs_t.append(tf)
    # Gather the expectation values from the results array
    for j in range(len(results_t[0].expect)):
        nn_exp = np.array([])
        for i in range(len(results_t)):
            nn_exp = np.append(nn_exp, results_t[i].expect[j])
        if j == 0:
            n_exp_t = np.array([nn_exp])
        else:
            n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
    # Gather the times for the plots
    times_t = np.array([])
    for i in range(len(results_t)):
        times_t = np.append(times_t, results_t[i].times)
    pl=np.array([])
    pl=np.add(n_exp_t[3],n_exp_t[4])
    pl=np.add(pl,n_exp_t[5])
    if c==0:
        pls = np.array([pl])
        c+=1
    else:
        pls = np.append(pls,[pl],axis=0)
print(f"Calculations are finished!! See the plots below")
# Set the colors of each state
colors = [
    "dodgerblue",
    "chocolate",
    "gold",
    "mediumpurple",
    "mediumseagreen",
    "lightskyblue",
    "magenta",
    "mediumvioletred",
    "forestgreen",
]
# Defines figure size
plt.figure(figsize=(14, 8))
a=0
state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
for pl in pls:
    plt.plot(times_t[-5200:],pl[-5200:], color=colors[a], label=f"init state=${state[a]}$")
    a+=1

eps_y = abs(np.min(pls) - np.max(pls))*5e-2
eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
plt.ylabel(f"PL(a.u.)",fontsize=20)
plt.xlabel(r"Time($\mu$s)",fontsize=20)

# set the time limits you want to show in the plot
t_lim = (times_t[-5200]-eps_x, times_t[-1] + eps_x)
plt.xlim(t_lim)
plt.ylim((np.min(pls) - eps_y, np.max(pls) + eps_y))
plt.minorticks_on()

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))
a=0
state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
for pl in pls_pure:
    plt.plot(times_t_pure[0:],pl[0:], ls='--', color=colors[a], label=f"init Pure state=${state[a]}$")
    a+=1
a=0
for pl in pls:
    plt.plot(times_t[-5200:]-11.2,pl[-5200:], color=colors[a], label=f"init state=${state[a]}$")
    a+=1
eps_y = abs(np.min(pls_pure) - np.max(pls_pure))*5e-2
eps_x = abs(np.min(times_t[-5200:]) - np.max(times_t[-5200:]))*5e-2
plt.ylabel(f"PL(a.u.)",fontsize=20)
plt.xlabel(r"Time($\mu$s)",fontsize=20)

# set the time limits you want to show in the plot
t_lim = (times_t_pure[0]-eps_x, times_t_pure[-1] + eps_x)
plt.xlim(t_lim)
plt.ylim((np.min(pls_pure) - eps_y, np.max(pls_pure) + eps_y))
plt.minorticks_on()

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.show()

# Modified Model with Pulsed Lasers

## Introduction
...

## Mathematical Formulation
...

## Simulation Code

### Continuous laser protocol

In [ ]:
def continuous_mg(laser_time, mw_time, free_time,om_r=Om_r,
    w_p=W_p,om=omega,protocol_repeats=1,progress_bar="ON"):
    """
    Perform continuous calculations for a multi-level quantum system.
    Args:
        laser_time (float): Time interval for laser evolution.
        mw_time (float): Time interval for microwave evolution.
        free_time (float): Time interval for free evolution.
        om_r (float): Rabi frequency for the system (default: Om_r).
        w_p (float): Transition frequency for the system (default: W_p).
        args (dict): Additional arguments for the system (default: {"w": omega}).
        protocol_repeats (int): Number of times to repeat the protocol (default: 1).
        progress_bar (str): Option to display progress bar ("ON" or "OFF") (default: "ON").
    Returns:
        tuple: A tuple containing the following elements:
            - n_exp (ndarray): Expectation values for the system.
            - times (ndarray): Time points for the evolution.
            - tis (list): List of activation times for laser and microwaves.
            - tfs (list): List of deactivation times for laser and microwaves.
            - results (ndarray): Array of Result class instances.
    Raises:
        None.
    """      
    # Initial state where the ground state is evenly populated
    init_state = 1 / 3 * (n1 + n2 + n3)

    # Array to collect the result Class
    results = np.array([])

    # Initial time of the evolution
    ti = 0.0

    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis = []
    tfs = []

    # Set the number of times to repeat the protocol
    n = protocol_repeats
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = laser_time
    dt_free = free_time
    dt_mw = mw_time
    print("Starting continuous calculations, this may take a while...")
    # This loop is defined to reproduce the same protocol as a pulsed protocol, but with continuous laser
    for i in tqdm(range(n)):
        tis.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg(dt_laser,
                                 init_state,
                                 ti=ti,
                                 mode="Laser",
                                 om_r=om_r,
                                 w_p=w_p,
                                 om=om,
                                 progress_bar=progress_bar,
                                 i=i) # type:ignore
        # Store the Result class for when the Laser is on
        results = np.append(results, result) # type: ignore
        # Save the initial state for the next part of the protocol
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg(dt_free,
                                 init_state,
                                 ti=ti,
                                 mode="Free",
                                 om_r=om_r,
                                 w_p=w_p,
                                 om=om,
                                 progress_bar=progress_bar,
                                 i=i) # type:ignore
        # Store the Result class for MW on evolution
        results = np.append(results, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg(dt_mw,
                                 init_state,
                                 ti=ti,
                                 mode="MW",
                                 om_r=om_r,
                                 w_p=w_p,
                                 om=om,
                                 progress_bar=progress_bar,
                                 i=i) # type:ignore
        # Store the Result class for MW on evolution
        results = np.append(results, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
    # Gather the expectation values from the results array
    for j in range(len(results[0].expect)):
        nn_exp = np.array([])
        for i in range(len(results)):
            nn_exp = np.append(nn_exp, results[i].expect[j])
        if j == 0:
            n_exp = np.array([nn_exp])
        else:
            n_exp = np.append(n_exp, [nn_exp], axis=0) # type: ignore
    if len(n_exp)>8:
        s_exp=n_exp[len(n_exp)-6:]
        n_exp=n_exp[:len(n_exp)-6]
    # Gather the times for the plots
    times = np.array([])
    for i in range(len(results)):
        times = np.append(times, results[i].times)
    print("Calculations are finished!! param order(Expect,times,tis,tfs,results)")
    return n_exp, times, tis, tfs, results,s_exp

### Pulsed laser protocol

In [ ]:
def pulsed_mg(active_laser_time, delay, total_pulse_time, mw_time, free_time, om_r=Om_r, w_p=W_p, om=omega, protocol_repeats=1, progress_bar="ON"):
    """
    Perform pulsed laser and microwave evolution calculations for a multi-level quantum system.
    Parameters:
    active_laser_time (float): Time duration for which the laser is active during each pulse.
    delay (float): Time delay between consecutive laser pulses.
    total_pulse_time (float): Total time duration for the pulsed laser evolution.
    mw_time (float): Time duration for which the microwaves are active.
    free_time (float): Time duration for free evolution between consecutive pulses.
    om_r (float): Rabi frequency of the MW.
    w_p (float): Frequency of the laser.
    args (dict): Additional arguments for the dynamics calculation.
    protocol_repeats (int): Number of times to repeat the protocol.
    progress_bar (str): Option to display progress bar during calculations. Can be "ON" or "OFF".
    Returns:
    tuple: A tuple containing the following elements:
        - n_p_exp (ndarray): Expectation values of the quantum system at each time step.
        - times_p (ndarray): Array of time values for the evolution.
        - tis (list): List of initial times for each part of the protocol.
        - tfs (list): List of final times for each part of the protocol.
        - active_laser_time (float): Time duration for which the laser is active during each pulse.
        - delay (float): Time delay between consecutive laser pulses.
        - total_pulse_time (float): Total time duration for the pulsed laser evolution.
        - results_p (ndarray): Array of Result class instances for each part of the protocol.
    """
    # Initial state where the ground state is evenly populated
    init_state = 1 / 3 * (n1 + n2 + n3)

    # Array to collect the result Class
    results_p = np.array([])

    # Initial time of the evolution
    ti = 0.0

    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis = []
    tfs = []

    # Set the number of times to repeat the protocol
    m = protocol_repeats
    # Set the time intervals for each part of the evolution (with pulse laser, free evolution of pulsed laser , with microwaves and free evolution)
    dt_laser = active_laser_time
    dt_free_pulse = delay
    t_laser_pulse = total_pulse_time
    dt_mw = mw_time
    dt_free = free_time
    # Set the number of times to repeat the pulsed laser
    n = int(t_laser_pulse // (dt_laser + dt_free_pulse))
    print("Starting pulse laser calculations, this may take a while...")
    # This loop must be defined in a way to reproduce the protocol choosen
    for j in tqdm(range(m),desc="Protocol"):
        tis.append(ti)
        for i in tqdm(range(n),desc="Pulse",leave=False):
            # Here Laser is ON
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_mg(dt_laser,
                                     init_state,
                                     ti=ti,
                                     mode="Laser",
                                     om_r=om_r,
                                     w_p=w_p,
                                     om=om,
                                     progress_bar="OFF",
                                     i=i) # type:ignore
            # Store the Result class for when the Laser is on
            results_p = np.append(results_p, result) # type: ignore
            # Make initial time to be the end time of the last part of the protocol
            ti = tf
            # Make the last state  of this evolution step be the initial states of  the next iteration
            init_state = result.states[-1]
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_mg(dt_free_pulse,
                                     init_state,
                                     ti=ti,
                                     mode="Free",
                                     om_r=om_r,
                                     w_p=w_p,
                                     om=om,
                                     progress_bar="OFF",
                                     i=i) # type:ignore
            # Store the Result class for metastable state depletion evolution
            results_p = np.append(results_p, result) # type: ignore
            # Save the initial state for the next part of the protocol
            init_state = result.states[-1]
            # Make the initial time the end time of the previous part of the procotol
            ti = tf
        tfs.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg(dt_free,
                                 init_state,
                                 ti=ti,
                                 mode="Free",
                                 om_r=om_r,
                                 w_p=w_p,
                                 om=om,
                                 progress_bar=progress_bar,
                                 i=j) # type:ignore
        # Store the Result class for MW on evolution
        results_p = np.append(results_p, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the  next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg(dt_mw,
                                 init_state,
                                 ti=ti,
                                 mode="MW",
                                 om_r=om_r,
                                 w_p=w_p,
                                 om=om,
                                 progress_bar=progress_bar,
                                 i=j) # type:ignore
        # Store the Result class for MW on evolution
        results_p = np.append(results_p, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
    # Gather the expectation values from the results array
    for j in range(len(results_p[0].expect)):
        nn_exp = np.array([])
        for i in range(len(results_p)):
            nn_exp = np.append(nn_exp, results_p[i].expect[j])
        if j == 0:
            n_p_exp = np.array([nn_exp])
        else:
            n_p_exp = np.append(n_p_exp, [nn_exp], axis=0) # type: ignore
    if len(n_p_exp)>8:
        s_p_exp=n_p_exp[len(n_p_exp)-6:]
        n_p_exp=n_p_exp[:len(n_p_exp)-6]
    # Gather the times for the plots
    times_p = np.array([])
    for i in range(len(results_p)):
        times_p = np.append(times_p, results_p[i].times)
    print(
        "Calculations are finished!! param order(Expect,times,tis,tfs,active_laser_time,delay,total_pulse_time,results_p)"
    )
    return (
        n_p_exp,
        times_p,
        tis,
        tfs,
        active_laser_time,
        delay,
        total_pulse_time,
        results_p,
        s_p_exp
    )

## Results and Discussion
...

In [ ]:
protocol_1 = pulsed_mg(0.4, 0.6, 100, 0.0, 0.0)
cont_1 = continuous_mg(100, 0.0, 0.0)

protocol_2 = pulsed_mg(0.6, 0.4, 100, 0.0, 0.0)
cont_2 = continuous_mg(100, 0.0, 0.0)

### Plot functions

In [ ]:
def plot_pulsed(
    n_puls_exp,
    times_puls,
    tis,
    tfs,
    laser_t,
    delay,
    total_pulse_time,
):
    """
    Plot the population of each state over time.

    Args:
        n_puls_exp (list): List of population values for each state in the pulsed laser experiment.
        times_puls (list): List of time values for the pulsed laser experiment.
        tis (list): List of start times for laser/microwave pulses.
        tfs (list): List of end times for laser/microwave pulses.
        laser_t (float): Laser duration in microseconds.
        delay (float): Delay time in microseconds.
        total_pulse_time (float): Total pulse time in microseconds.

    Returns:
        None
    """
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "gold",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    for i in range(len(n_puls_exp) - 1):
        plt.plot(times_puls, n_puls_exp[i], label=f"Pulsed $n_{i+1}$", color=colors[i])
    plt.plot(times_puls, n_puls_exp[-1], label="Pulsed $n_c$", color=colors[-1])

    # Highlighting the times where the laser and microwaves are on and the  metastable state depletion evolutions
    n = 0
    eps_y = 0.03
    eps_x = 0.2
    for i in zip(tis, tfs):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(n_puls_exp) - eps_y, np.max(n_puls_exp) + eps_y],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(n_puls_exp) - eps_y, np.max(n_puls_exp) + eps_y],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MS     depl #{int(n/3)+1}",
                )
            else:
                plt.fill_betweenx(
                    [np.min(n_puls_exp) - eps_y, np.max(n_puls_exp) + eps_y],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MW ON #   {int(n/3)+1}",
                )
        n += 1
    plt.ylabel(f"Population",fontsize=20)
    plt.xlabel(r"Time($\mu$s)",fontsize=20)
    plt.title(
        f"Laser:{laser_t} $\\mu$s, Delay:{delay} $\\mu$s, Pulse time:{total_pulse_time} $\\mu$s",fontsize=20
    )
    # set the time limits you want to show in the plot
    t_lim = (-eps_x, times_puls[-1] + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(n_puls_exp) - eps_y, np.max(n_puls_exp) + eps_y))
    plt.minorticks_on()
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.show()

In [ ]:
def plot_pulsed_v_cont(
    n_puls_exp,
    times_puls,
    tis,
    tfs,
    laser_t,
    delay,
    total_pulse_time,
    n_cont_exp,
    times_cont,
):
    """
    Plot the population of each state over time.

    Args:
        n_puls_exp (list): List of population values for each state in the pulsed laser experiment.
        times_puls (list): List of time values for the pulsed laser experiment.
        tis (list): List of start times for laser/microwave pulses.
        tfs (list): List of end times for laser/microwave pulses.
        laser_t (float): Laser duration in microseconds.
        delay (float): Delay time in microseconds.
        total_pulse_time (float): Total pulse time in microseconds.
        n_cont_exp (list): List of population values for each state in the continuous laser experiment.
        times_cont (list): List of time values for the continuous laser experiment.

    Returns:
        None
    """
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "gold",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    for i in range(len(n_puls_exp) - 1):
        plt.plot(times_puls, n_puls_exp[i], label=f"Pulsed $n_{i+1}$", color=colors[i])
    plt.plot(times_puls, n_puls_exp[-1], label="Pulsed $n_c$", color=colors[-1])
    for i in range(len(n_cont_exp) - 1):
        plt.plot(
            times_cont,
            n_cont_exp[i],
            label=f"Cont $n_{i+1}$",
            color=colors[i],
            ls="--",
        )
    plt.plot(
        times_cont, n_cont_exp[-1], label="Cont $n_c$", color=colors[-1], ls="--"
    )

    # Highlighting the times where the laser and microwaves are on and the  metastable state depletion evolutions
    n = 0
    eps_y = 0.03
    eps_x = 0.2
    for i in zip(tis, tfs):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(np.append(n_puls_exp,n_cont_exp)) - eps_y, np.max(np.append(n_puls_exp,n_cont_exp)) + eps_y],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(np.append(n_puls_exp,n_cont_exp)) - eps_y, np.max(np.append(n_puls_exp,n_cont_exp)) + eps_y],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MS     depl #{int(n/3)+1}",
                )
            else:
                plt.fill_betweenx(
                    [np.min(np.append(n_puls_exp,n_cont_exp)) - eps_y, np.max(np.append(n_puls_exp,n_cont_exp)) + eps_y],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MW ON #   {int(n/3)+1}",
                )
        n += 1
    plt.ylabel(f"Population",fontsize=20)
    plt.xlabel(r"Time($\mu$s)",fontsize=20)
    plt.title(
        f"Laser:{laser_t:.2f} $\\mu$s, Delay:{delay:.2f} $\\mu$s, Pulse time:{total_pulse_time:.2f} $\\mu$s",fontsize=20
    )
    # set the time limits you want to show in the plot
    t_lim = (-eps_x, times_puls[-1] + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(n_puls_exp,n_cont_exp)) - eps_y, np.max(np.append(n_puls_exp,n_cont_exp)) + eps_y))
    plt.minorticks_on()
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.show()

In [ ]:
def plot_pulsed_v_pulsed(
    n_puls_exp_1,
    times_puls_1,
    tis_1,
    tfs_1,
    laser_t_1,
    delay_1,
    total_pulse_time_1,
    n_puls_exp_2,
    times_puls_2,
    tis_2,
    tfs_2,
    laser_t_2,
    delay_2,
    total_pulse_time_2,
):
    """
    Plot the population of each state for two pulsed experiments.

    Parameters:
    - n_puls_exp_1 (list): List of population values for each state in the first pulsed experiment.
    - times_puls_1 (list): List of time values for the first pulsed experiment.
    - tis_1 (list): List of start times for laser/microwave on/off intervals in the first pulsed experiment.
    - tfs_1 (list): List of end times for laser/microwave on/off intervals in the first pulsed experiment.
    - laser_t_1 (float): Laser time for the first pulsed experiment.
    - delay_1 (float): Delay time for the first pulsed experiment.
    - total_pulse_time_1 (float): Total pulse time for the first pulsed experiment.
    - n_puls_exp_2 (list): List of population values for each state in the second pulsed experiment.
    - times_puls_2 (list): List of time values for the second pulsed experiment.
    - tis_2 (list): List of start times for laser/microwave on/off intervals in the second pulsed experiment.
    - tfs_2 (list): List of end times for laser/microwave on/off intervals in the second pulsed experiment.
    - laser_t_2 (float): Laser time for the second pulsed experiment.
    - delay_2 (float): Delay time for the second pulsed experiment.
    - total_pulse_time_2 (float): Total pulse time for the second pulsed experiment.

    Returns:
    None
    """
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "gold",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    for i in range(len(n_puls_exp_1) - 1):
        plt.plot(
            times_puls_1, n_puls_exp_1[i], label=f"Pulsed$_1$ $n_{i+1}$", color=colors[i]
        )
    plt.plot(times_puls_1, n_puls_exp_1[-1], label="Pulsed$_1$ $n_c$", color=colors[-1])
    colors = colors[::-1]
    for i in range(len(n_puls_exp_2) - 1):
        plt.plot(
            times_puls_2,
            n_puls_exp_2[i],
            label=f"Pulsed$_2$ $n_{i+1}$",
            color=colors[i],
            ls="--",
        )
    plt.plot(
        times_puls_2,
        n_puls_exp_2[-1],
        label="Pulsed$_2$ $n_c$",
        color=colors[-1],
        ls="--",
    )

    # Highlighting the times where the laser and microwaves are on and the  metastable state depletion evolutions
    n = 0
    eps_y = 0.03
    eps_x = 0.2
    for i in zip(tis_1, tfs_1):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(np.append(n_puls_exp_1,n_puls_exp_2)) - eps_y, np.max(np.append(n_puls_exp_1,n_puls_exp_2)) + eps_y],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(np.append(n_puls_exp_1,n_puls_exp_2)) - eps_y, np.max(np.append(n_puls_exp_1,n_puls_exp_2)) + eps_y],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MS     depl #{int(n/3)+1}",
                )
            else:
                plt.fill_betweenx(
                    [np.min(np.append(n_puls_exp_1,n_puls_exp_2)) - eps_y, np.max(np.append(n_puls_exp_1,n_puls_exp_2)) + eps_y],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MW ON #   {int(n/3)+1}",
                )
        n += 1
    plt.ylabel(f"Population",fontsize=20)
    plt.xlabel(r"Time($\mu$s)",fontsize=20)
    plt.title(
        f"Laser$_1$:{laser_t_1} $\\mu$s, Delay$_1$:{delay_1:.2f} $\\mu$s, Pulse time$_1$:{total_pulse_time_1:.2f} $\\mu$s\nLaser$_2$:{laser_t_2:.2f} $\\mu$s, Delay$_2$:{delay_2:.2f} $\\mu$s, Pulse time$_2$:{total_pulse_time_2:.2f} $\\mu$s",fontsize=20
    )
    # set the time limits you want to show in the plot
    t_lim = (-eps_x, times_puls_1[-1] + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(n_puls_exp_1,n_puls_exp_2)) - eps_y, np.max(np.append(n_puls_exp_1,n_puls_exp_2)) + eps_y))
    plt.minorticks_on()
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.show()

### Plots

In [ ]:
plot_pulsed_v_cont(
    protocol_1[0],
    protocol_1[1],
    protocol_1[2],
    protocol_1[3],
    protocol_1[4],
    protocol_1[5],
    protocol_1[6],
    cont_1[0],
    cont_1[1],
)

In [ ]:
plot_pulsed_v_cont(
    protocol_2[0],
    protocol_2[1],
    protocol_2[2],
    protocol_2[3],
    protocol_2[4],
    protocol_2[5],
    protocol_2[6],
    cont_2[0],
    cont_2[1],
)

In [ ]:
plot_pulsed_v_pulsed(
    protocol_1[0],
    protocol_1[1],
    protocol_1[2],
    protocol_1[3],
    protocol_1[4],
    protocol_1[5],
    protocol_1[6],
    protocol_2[0],
    protocol_2[1],
    protocol_2[2],
    protocol_2[3],
    protocol_2[4],
    protocol_2[5],
    protocol_2[6],
)

# Model with Hyperfine Interaction

## Introduction
...

## Mathematical Formulation
$$
H^j_{hf}=A^j_\parallel S^j_z I_z + A^j_\perp (S^j_x I_x + S^j_y I_y) \\
H^j=H^j_{mg}\otimes\mathbb{I}_N+H^j_{hf} \\
j=\{gs,es\}\\
A^{gs}_\parallel=3.03 \ \text{MHz} \ \text{and} \ A^{gs}_\perp=3.65 \ \text{MHz} \\
A^{es}_\parallel=-57.8 \ \text{MHz} \ \text{and} \ A^{es}_\perp=-39.2 \ \text{MHz} \\
$$

Relaxation and dephasing terms
$$
L_{T_{1,N}}= \sqrt{\gamma^{N}_1/2} \ \sigma^{N}_-\\
L_{T^*_{2,N}}= \sqrt{\gamma^{N}_2} \ \sigma^{N}_z\\
N=\{^{14}N \ \text{or}  \ ^{15}N\} \\
T_{1,N} \approx 10^5 \ \mu s, \ T_{2,N} \approx 10^3 \ \mu s
$$

## Simulation Code

### Hamiltonian and constants

In [ ]:
#Dimention of the Hilbert space
dim = 7

t1_n=1e5
t2_n=1e3
# Constants (MHz)
gamma_n=1/t1_n,1/t2_n
a_gs=3.03,3.65
a_es=-57.8,-39.2
mu_n = 431.7*1e-6 # (MHz/G)
mu_e = 2.8 # (Mhz/G)

# Nitrogen Nucleous
IdN15 = qt.qeye(2)
nit = qt.basis(2, 0), qt.basis(2, 1)
sx_n, sy_n, sz_n = qt.sigmax()*0.5, qt.sigmay()*0.5, qt.sigmaz()*0.5
sm_n,sp_n=sx_n-1j*sy_n,sx_n+1j*sy_n
S_n=np.array([sx_n,sy_n,sz_n])

def H_mg_hf(om_r):
    """Returns the Hamiltonian of the system based on whether the MW is on or off
    Parameters:
        om_r (float) - Rabi frequency

    Returns:
        H_0 (list) - list of the Hamiltonian terms and their time dependence
    """
    H_0 = [[(0.5 * om_r * (ground[1]*ground[2].dag()))&IdN15, "exp(1j*w*t)"],  #type: ignore
           [(0.5 * om_r * (ground[2]*ground[1].dag()))&IdN15, "exp(-1j*w*t)"], #type: ignore
           a_gs[0]*(sz_gs&sz_n) + a_gs[1]*((sx_gs&sx_n) + (sy_gs&sy_n)),
           a_es[0]*(sz_es&sz_n) + a_es[1]*((sx_es&sx_n) + (sy_es&sy_n))]
    H_n=[[IdNV&(0.5*om_r*(mu_n/mu_e)*(nit[0]*nit[1].dag())),"exp(1j*w*t)"],   #type: ignore
          [IdNV&(0.5*om_r*(mu_n/mu_e)*(nit[1]*nit[0].dag())),"exp(-1j*w*t)"]] #type: ignore
    return [*H_0 , *H_n]

def L_mg_hf(w_p,k_index=k_ind):
    """Returns the Lindblad operators of the system
    Parameters:
        w_p (float) - Laser pump rate
    Returns:
        c_ops (list) - list of the Lindblad operators
    """
    K_s=[[66.0,0.0,57.0,1.0,0.7],
         [77.0,0.0,30.0,3.3,0.0],
         [62.7,12.97,80.0,3.45,1.08],
         [63.2,10.8,60.7,0.8,0.4],
         [67.4,9.9,96.6,4.83,1.055]]
    k41 = K_s[k_index][0]
    k52 = K_s[k_index][0]
    k63 = K_s[k_index][0]
    k57 = K_s[k_index][2]
    k67 = K_s[k_index][2]
    k47 = K_s[k_index][1]
    k71 = K_s[k_index][3]
    k72 = K_s[k_index][4]
    k73 = K_s[k_index][4]
    
    c_ops = []

    c_ops.append((np.sqrt(w_p) * (excited[1] * ground[1].dag()))&IdN15)  # n1 to n4 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[2] * ground[2].dag()))&IdN15)  # n2 to n5 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[0] * ground[0].dag()))&IdN15) # n3 to n6 #type: ignore

    c_ops.append((np.sqrt(k41) * (ground[1] * excited[1].dag()))&IdN15)  # n4 to n1 #type: ignore
    c_ops.append((np.sqrt(k71) * (ground[1] * isc.dag()))&IdN15)  # n7 to n1 #type: ignore

    c_ops.append((np.sqrt(k52) * (ground[2] * excited[2].dag()))&IdN15)  # n5 to n2 #type: ignore
    c_ops.append((np.sqrt(k72) * (ground[2] * isc.dag()))&IdN15)  # n7 to n2 #type: ignore

    c_ops.append((np.sqrt(k63) * (ground[0] * excited[0].dag()))&IdN15)  # n6 to n3 #type: ignore
    c_ops.append((np.sqrt(k73) * (ground[0] * isc.dag()))&IdN15)  # n7 to n3 #type: ignore

    c_ops.append((np.sqrt(k47) * (isc * excited[1].dag()))&IdN15)  # n4 to n7 #type: ignore
    c_ops.append((np.sqrt(k57) * (isc * excited[2].dag()))&IdN15)  # n5 to n7 #type: ignore
    c_ops.append((np.sqrt(k67) * (isc * excited[0].dag()))&IdN15)  # n6 to n7 #type: ignore
    # Add collapse operators for decoherence   
    c_ops.append((np.sqrt(gamma_gs[1]) * sz_gs)&IdN15)
    c_ops.append((np.sqrt(gamma_gs[0]/2) * (sm_gs))&IdN15)
    c_ops.append((np.sqrt(gamma_gs[0]/2) * (sp_gs))&IdN15)
    c_ops.append((np.sqrt(gamma_es[1]) * sz_es)&IdN15)
    c_ops.append((np.sqrt(gamma_es[0]/2) * (sm_es))&IdN15)
    c_ops.append((np.sqrt(gamma_es[0]/2) * (sp_es))&IdN15)
    c_ops.append(IdNV&(np.sqrt(gamma_n[1]) * sz_n))
    c_ops.append(IdNV&(np.sqrt(gamma_n[0]/2) * (sm_n)))
    c_ops.append(IdNV&(np.sqrt(gamma_n[0]/2) * (sp_n)))

    return c_ops

def dynamics_mg_hf(
    dt,
    init_state,
    om_r=None,
    om=None,
    w_p=None,
    k_index=k_ind,
    exp_ops=None,
    ti=0.0,
    mode="Free",
    progress_bar="ON",
    i=0,
):
    """
    Simulate the dynamics of a quantum system under hyperfine interaction using the Hamiltonian and collapse operators.
    including optical transition rates index.
    Where, when using k_index=2 -> K_s[k_index]=[62.7,12.97,80.0,3.45,1.08], and the full K_s list is:
        K_s=[[66.0,0.0,57.0,1.0,0.7],
             [77.0,0.0,30.0,3.3,0.0],
             [62.7,12.97,80.0,3.45,1.08],
             [63.2,10.8,60.7,0.8,0.4],
             [67.4,9.9,96.6,4.83,1.055]]
    Parameters:
    - dt (float): Total simulation time.
    - init_state (qutip.Qobj): Initial quantum state of the system.
    - om_r (float, optional): Rabi frequency for microwave interactions. Defaults to Om_r.
    - om (float, optional): Angular frequency of the system. Defaults to omega.
    - w_p (float, optional): Laser frequency. Defaults to W_p.
    - k_index(int, optional): Index for the optical transition rates. Defaults to k.
    - exp_ops (list of qutip.Qobj, optional): List of operators for which to compute expectation values. 
        Defaults to a list of tensor products including different Pauli operators and state projectors.
    - ti (float, optional): Initial time of the simulation. Defaults to 0.0.
    - mode (str, optional): Simulation mode. Can be "Free", "MW", "Laser", or "Laser-MW". Defaults to "Free".
    - progress_bar (str, optional): Option to display a progress bar. Can be "ON" or "OFF". Defaults to "ON".
    - i (int, optional): Counter for the progress bar. Defaults to 0.

    Returns:
    - tf (float): Final time of the simulation.
    - result (qutip.solver.Result): Result object containing the simulation output.
    """
    # Default values
    if om_r is None: om_r = Om_r
    if om is None: om = omega
    if w_p is None: w_p = W_p
    if exp_ops is None: exp_ops = [
        n1&IdN15, n2&IdN15, n3&IdN15,
        n4&IdN15, n5&IdN15, n6&IdN15,
        n7&IdN15, nc&IdN15,
        sx_gs&IdN15, sy_gs&IdN15, sz_gs&IdN15,
        sx_es&IdN15, sy_es&IdN15, sz_es&IdN15,
        IdNV&nit[0] * nit[0].dag(), IdNV&nit[1] * nit[1].dag() #type: ignore
    ]

    # Time resolution based on dt
    t_bins = 1000 if dt <= 5 else 5000
    
    # Define Hamiltonian and collapse operators based on mode
    match mode:
        case "Free":
            H = H_mg_hf(0.0)
            c_ops = L_mg_hf(0.0, k_index=k_index)
        case "MW":
            H = H_mg_hf(om_r)
            c_ops = L_mg_hf(0.0, k_index=k_index)
        case "Laser":
            H = H_mg_hf(0.0)
            c_ops = L_mg_hf(w_p, k_index=k_index)
        case "Laser-MW":
            H = H_mg_hf(om_r)
            c_ops = L_mg_hf(w_p, k_index=k_index)
        case _:
            raise ValueError('mode must be one of "Free", "MW", "Laser", or "Laser-MW"')

    # Arguments for the Hamiltonian
    args = {"w": om}
    
    tf = ti + dt

    # Solve the master equation
    match progress_bar:
        case "OFF":
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                e_ops=exp_ops,
                args=args,
                options={"store_states": True},
            )
        case "ON":
            print(f"{mode} {int(i + 1)} \n ti | tf \n {int(ti)} | {int(tf)}")
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                e_ops=exp_ops,
                args=args,
                options={"store_states": True, "progress_bar": "tqdm"},
            )
        case _:
            raise ValueError('progress_bar must be "ON" or "OFF"')

    return tf, result

### Continuos laser dynamics function

In [ ]:
def continuous_mg_hf(laser_time, mw_time, free_time,om_r=Om_r,
                     w_p=W_p,om = omega,protocol_repeats=1,progress_bar="ON"):
    """
    Simulates the evolution of a quantum system under a given protocol with continuos laser.

    Args:
        laser_time (float): The duration of the laser pulse in each iteration.
        mw_time (float): The duration of the microwave pulse in each iteration.
        free_time (float): The duration of the free evolution in each iteration.
        protocol_repeats (int, optional): The number of times to repeat the protocol. Defaults to 1.

    Returns:
        tuple: A tuple containing the following arrays:
            - n_exp (numpy.ndarray): An array of expectation values for different operators.
            - times (numpy.ndarray): An array of time values.
            - tis (numpy.ndarray): An array of initial times for each part of the protocol.
            - tfs (numpy.ndarray): An array of final times for each part of the protocol.
            - results (numpy.ndarray): An array of the Result class object for the time evolution.
    """
    # Initial state where the ground state is evenly populated
    init_state = (1 / 3 * (n1 + n2 + n3))&(1/2*IdN15)

    # Array to collect the result Class
    results = np.array([])

    # Initial time of the evolution
    ti = 0.0

    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis = []
    tfs = []

    # Set the number of times to repeat the protocol
    n = protocol_repeats
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = laser_time
    dt_free = free_time
    dt_mw = mw_time
    print("Starting continuous calculations, this may take a while...")
    # This loop is defined to reproduce the same protocol as a pulsed protocol, but with continuous laser
    for i in tqdm(range(n)):
        tis.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg_hf(dt_laser,
                                    init_state,
                                    ti=ti,
                                    mode="Laser",
                                    om_r=om_r,
                                    w_p=w_p,
                                    om = om,
                                    progress_bar=progress_bar,
                                    i=i) # type:ignore
        # Store the Result class for when the Laser is on
        results = np.append(results, result) # type: ignore
        # Save the initial state for the next part of the protocol
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg_hf(dt_free,
                                    init_state,
                                    ti=ti,
                                    mode="Free",
                                    om_r=om_r,
                                    w_p=w_p,
                                    om=om,
                                    progress_bar=progress_bar,
                                    i=i) # type:ignore
        # Store the Result class for MW on evolution
        results = np.append(results, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg_hf(dt_mw,
                                    init_state,
                                    ti=ti,
                                    mode="MW",
                                    om_r=om_r,
                                    w_p=w_p,
                                    om=om,
                                    progress_bar=progress_bar,
                                    i=i) # type:ignore
        # Store the Result class for MW on evolution
        results = np.append(results, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
    # Gather the expectation values from the results array
    for j in range(len(results[0].expect)):
        nn_exp = np.array([])
        for i in range(len(results)):
            nn_exp = np.append(nn_exp, results[i].expect[j])
        if j == 0:
            n_exp = np.array([nn_exp])
        else:
            n_exp = np.append(n_exp, [nn_exp], axis=0) # type: ignore
    if len(n_exp)>8:
        s_exp=n_exp[len(n_exp)-8:len(n_exp)-2]
        nit_exp=n_exp[len(n_exp)-2:]
        n_exp=n_exp[:len(n_exp)-8]
    # Gather the times for the plots
    times = np.array([])
    for i in range(len(results)):
        times = np.append(times, results[i].times)
    print("Calculations are finished!! param order(Expect,times,tis,tfs)")
    return n_exp, times, tis, tfs, s_exp, nit_exp

### Pulsed laser dynamics function

In [ ]:
def pulsed_mg_hf(active_laser_time, delay, total_pulse_time, mw_time, free_time, om_r=Om_r, w_p=W_p, om=omega, protocol_repeats=1, progress_bar="ON"):
    """
    Perform pulsed laser and microwave evolution calculations for a multi-level quantum system.
    Parameters:
    active_laser_time (float): Time duration for which the laser is active during each pulse.
    delay (float): Time delay between consecutive laser pulses.
    total_pulse_time (float): Total time duration for the pulsed laser evolution.
    mw_time (float): Time duration for which the microwaves are active.
    free_time (float): Time duration for free evolution between consecutive pulses.
    om_r (float): Rabi frequency of the MW.
    w_p (float): Frequency of the laser.
    args (dict): Additional arguments for the dynamics calculation.
    protocol_repeats (int): Number of times to repeat the protocol.
    progress_bar (str): Option to display progress bar during calculations. Can be "ON" or "OFF".
    Returns:
    tuple: A tuple containing the following elements:
        - n_p_exp (ndarray): Expectation values of the quantum system at each time step.
        - times_p (ndarray): Array of time values for the evolution.
        - tis (list): List of initial times for each part of the protocol.
        - tfs (list): List of final times for each part of the protocol.
        - active_laser_time (float): Time duration for which the laser is active during each pulse.
        - delay (float): Time delay between consecutive laser pulses.
        - total_pulse_time (float): Total time duration for the pulsed laser evolution.
        - results_p (ndarray): Array of Result class instances for each part of the protocol.
    """
    # Initial state where the ground state is evenly populated
    init_state = (1 / 3 * (n1 + n2 + n3))&(1/2*IdN15)

    # Array to collect the result Class
    results_p = np.array([])

    # Initial time of the evolution
    ti = 0.0

    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis = []
    tfs = []

    # Set the number of times to repeat the protocol
    m = protocol_repeats
    # Set the time intervals for each part of the evolution (with pulse laser, free evolution of pulsed laser , with microwaves and free evolution)
    dt_laser = active_laser_time
    dt_free_pulse = delay
    t_laser_pulse = total_pulse_time
    dt_mw = mw_time
    dt_free = free_time
    # Set the number of times to repeat the pulsed laser
    n = int(t_laser_pulse // (dt_laser + dt_free_pulse))
    print("Starting pulse laser calculations, this may take a while...")
    # This loop must be defined in a way to reproduce the protocol choosen
    for j in tqdm(range(m),desc="Protocol"):
        tis.append(ti)
        for i in tqdm(range(n),desc="Pulse",leave=False):
            # Here Laser is ON
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_mg_hf(dt_laser,
                                     init_state,
                                     ti=ti,
                                     mode="Laser",
                                     om_r=om_r,
                                     w_p=w_p,
                                     om=om,
                                     progress_bar="OFF",
                                     i=i) # type:ignore
            # Store the Result class for when the Laser is on
            results_p = np.append(results_p, result) # type: ignore
            # Make initial time to be the end time of the last part of the protocol
            ti = tf
            # Make the last state  of this evolution step be the initial states of  the next iteration
            init_state = result.states[-1]
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_mg_hf(dt_free_pulse,
                                     init_state,
                                     ti=ti,
                                     mode="Free",
                                     om_r=om_r,
                                     w_p=w_p,
                                     om=om,
                                     progress_bar="OFF",
                                     i=i) # type:ignore
            # Store the Result class for metastable state depletion evolution
            results_p = np.append(results_p, result) # type: ignore
            # Save the initial state for the next part of the protocol
            init_state = result.states[-1]
            # Make the initial time the end time of the previous part of the procotol
            ti = tf
        tfs.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg_hf(dt_free,
                                 init_state,
                                 ti=ti,
                                 mode="Free",
                                 om_r=om_r,
                                 w_p=w_p,
                                 om=om,
                                 progress_bar=progress_bar,
                                 i=j) # type:ignore
        # Store the Result class for MW on evolution
        results_p = np.append(results_p, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the  next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg_hf(dt_mw,
                                 init_state,
                                 ti=ti,
                                 mode="MW",
                                 om_r=om_r,
                                 w_p=w_p,
                                 om=om,
                                 progress_bar=progress_bar,
                                 i=j) # type:ignore
        # Store the Result class for MW on evolution
        results_p = np.append(results_p, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
    # Gather the expectation values from the results array
    for j in range(len(results_p[0].expect)):
        nn_exp = np.array([])
        for i in range(len(results_p)):
            nn_exp = np.append(nn_exp, results_p[i].expect[j])
        if j == 0:
            n_p_exp = np.array([nn_exp])
        else:
            n_p_exp = np.append(n_p_exp, [nn_exp], axis=0) # type: ignore
    if len(n_p_exp)>8:
        s_p_exp=n_p_exp[len(n_p_exp)-8:len(n_p_exp)-2]
        nit_p_exp=n_p_exp[len(n_p_exp)-2:]
        n_p_exp=n_p_exp[:len(n_p_exp)-8]
    # Gather the times for the plots
    times_p = np.array([])
    for i in range(len(results_p)):
        times_p = np.append(times_p, results_p[i].times)
    print(
        "Calculations are finished!! param order(Expect,times,tis,tfs,active_laser_time,delay,total_pulse_time,results_p)"
    )
    return (
        n_p_exp,
        times_p,
        tis,
        tfs,
        active_laser_time,
        delay,
        total_pulse_time,
        results_p,
        s_p_exp,
        nit_p_exp
    )

## Results and Discussion
...

### Plot functions

In [ ]:
def plot_cont_hf_vs_cont(n_exp_hf,times_hf,tis_hf,tfs_hf,n_exp,times,tis,tfs):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "gold",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    hf="{hf}"
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    for i in range(len(n_exp_hf) - 1):
        plt.plot(times_hf, n_exp_hf[i], label=f"$n^{hf}_{i+1}$", color=colors[i])
    plt.plot(times_hf, n_exp_hf[-1], label="$n^{hf}_c$", color=colors[-1])
    for i in range(len(n_exp) - 1):
        plt.plot(times, n_exp[i], label=f"$n_{i+1}$", color=colors[i],ls='--')
    plt.plot(times, n_exp[-1], label="$n_c$", color=colors[-1],ls='--')
    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    n = 0
    eps = 5*1e-2
    for i in zip(tis, tfs):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(n_exp) - eps, np.max(n_exp) + eps],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
            t_laser=i[1]-i[0]
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(n_exp) - eps, np.max(n_exp) + eps],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MS depl #{int(n/3)+1}",
                )
                t_free=i[1]-i[0]
            else:
                plt.fill_betweenx(
                    [np.min(n_exp) - eps, np.max(n_exp) + eps],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MW ON #{int(n/3)+1}",
                )
                t_mw=i[1]-i[0]
        n += 1
    plt.ylabel(f"Population",fontsize=20)
    plt.xlabel(r"Time($\mu$s)",fontsize=20)
    plt.title(f"Comparisson between Hyperfine and No-HF evolution\n Laser:{t_laser:.2f} $\\mu$s, MW:{t_mw:.2f} $\\mu$s, Free:{t_free:.2f} $\\mu$s",fontsize=20)
    # set the time limits you want to show in the plot
    t_lim = (-5 * eps, tfs[-1] + 5 * eps)
    plt.xlim(t_lim)
    plt.ylim((np.min(n_exp) - eps, np.max(n_exp) + eps))
    plt.minorticks_on()
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.show()

In [ ]:
def plot_pulsed_hf_v_pulsed(
    n_puls_exp_1,
    times_puls_1,
    tis_1,
    tfs_1,
    laser_t_1,
    delay_1,
    total_pulse_time_1,
    n_puls_exp_2,
    times_puls_2,
    tis_2,
    tfs_2,
    laser_t_2,
    delay_2,
    total_pulse_time_2,
):
    """
    Plot the population of each state for two pulsed experiments.

    Parameters:
    - n_puls_exp_1 (list): List of population values for each state in the first pulsed experiment.
    - times_puls_1 (list): List of time values for the first pulsed experiment.
    - tis_1 (list): List of start times for laser/microwave on/off intervals in the first pulsed experiment.
    - tfs_1 (list): List of end times for laser/microwave on/off intervals in the first pulsed experiment.
    - laser_t_1 (float): Laser time for the first pulsed experiment.
    - delay_1 (float): Delay time for the first pulsed experiment.
    - total_pulse_time_1 (float): Total pulse time for the first pulsed experiment.
    - n_puls_exp_2 (list): List of population values for each state in the second pulsed experiment.
    - times_puls_2 (list): List of time values for the second pulsed experiment.
    - tis_2 (list): List of start times for laser/microwave on/off intervals in the second pulsed experiment.
    - tfs_2 (list): List of end times for laser/microwave on/off intervals in the second pulsed experiment.
    - laser_t_2 (float): Laser time for the second pulsed experiment.
    - delay_2 (float): Delay time for the second pulsed experiment.
    - total_pulse_time_2 (float): Total pulse time for the second pulsed experiment.

    Returns:
    None
    """
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "gold",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    hf="{hf}"
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    for i in range(len(n_puls_exp_1) - 1):
        plt.plot(
            times_puls_1, n_puls_exp_1[i], label=f"Pulsed$_{hf}$ $n_{i+1}$", color=colors[i]
        )
    plt.plot(times_puls_1, n_puls_exp_1[-1], label="Pulsed$_{hf}$ $n_c$", color=colors[-1])
    
    for i in range(len(n_puls_exp_2) - 1):
        plt.plot(
            times_puls_2,
            n_puls_exp_2[i],
            label=f"Pulsed $n_{i+1}$",
            color=colors[i],
            ls="--",
        )
    plt.plot(
        times_puls_2,
        n_puls_exp_2[-1],
        label="Pulsed $n_c$",
        color=colors[-1],
        ls="--",
    )

    # Highlighting the times where the laser and microwaves are on and the  metastable state depletion evolutions
    n = 0
    eps_y = 5*1e-2
    eps_x = 5*eps_y
    for i in zip(tis_1, tfs_1):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(np.append(n_puls_exp_1,n_puls_exp_2)) - eps_y, np.max(np.append(n_puls_exp_1,n_puls_exp_2)) + eps_y],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(np.append(n_puls_exp_1,n_puls_exp_2)) - eps_y, np.max(np.append(n_puls_exp_1,n_puls_exp_2)) + eps_y],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MS depl #{int(n/3)+1}",
                )
            else:
                plt.fill_betweenx(
                    [np.min(np.append(n_puls_exp_1,n_puls_exp_2)) - eps_y, np.max(np.append(n_puls_exp_1,n_puls_exp_2)) + eps_y],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MW ON #   {int(n/3)+1}",
                )
        n += 1
    plt.ylabel(f"Population",fontsize=20)
    plt.xlabel(r"Time($\mu$s)",fontsize=20)
    plt.title(f"Comparisson between Hyperfine and No-HF evolution \\w pulsed laser\n Laser:{laser_t_1:.2f} $\\mu$s, Delay:{delay_1:.2f} $\\mu$s, Pulse time:{total_pulse_time_1:.2f} $\\mu$s",fontsize=20
    )
    # set the time limits you want to show in the plot
    t_lim = (-eps_x, np.max([times_puls_1[-1],times_puls_2[-1]]) + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(n_puls_exp_1,n_puls_exp_2)) - eps_y, np.max(np.append(n_puls_exp_1,n_puls_exp_2)) + eps_y))
    plt.minorticks_on()
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.show()

### Plots

In [ ]:
cont_hf=continuous_mg_hf(10.0, 4.5, 1.0)
cont=continuous_mg(10.0,4.5,1.0)

puls_hf=pulsed_mg_hf(0.4, 0.6, 10, 4.5, 1.0)
puls=pulsed_mg(0.4, 0.6, 10, 4.5, 1.0)

In [ ]:
plot_continuous(cont_hf[0],cont_hf[1],cont_hf[2],cont_hf[3])

In [ ]:
plot_cont_hf_vs_cont(cont_hf[0],cont_hf[1],cont_hf[2],cont_hf[3],cont[0],cont[1],cont[2],cont[3])

In [ ]:
plot_pulsed(puls_hf[0],puls_hf[1],puls_hf[2],puls_hf[3],puls_hf[4],puls_hf[5],puls_hf[6])

In [ ]:
plot_pulsed_hf_v_pulsed(puls_hf[0],puls_hf[1],puls_hf[2],puls_hf[3],puls_hf[4],puls_hf[5],puls_hf[6],
                     puls[0],puls[1],puls[2],puls[3],puls[4],puls[5],puls[6])

In [ ]:
a,b=1,2
assert(a==b)

### PL calculation and plot
Do not run this unless you want to wait a lot

In [ ]:
t, n = 1.1, 54
if run_rabi:
    def pulsed_mg_hf_para(t):
        return pulsed_mg_hf(0.3,0.7,10.0,t,0.0,protocol_repeats=10,progress_bar="OFF")
    def pulsed_mg_para(t):
        return pulsed_mg(0.3,0.7,10.0,t,0.0,protocol_repeats=10,progress_bar="OFF")
    def rabi_para(time:float,n:int):
        flur_hf=np.array([])
        flur=np.array([])
        times=np.arange(time/n,time*(1+1/n),time/n)
        print("""This takes a lot depending on t_mw, the number inside range(), the first three terms in pulsed and
              the protocol_repeats.\nBe patient...\n\n """)
        pulses_hf=qt.loky_pmap(pulsed_mg_hf_para,times,progress_bar='tqdm',map_kw={'fail_fast':True})
        pulses=qt.loky_pmap(pulsed_mg_para,times,progress_bar='tqdm',map_kw={'fail_fast':True})
        for puls_hf,puls in tqdm(zip(pulses_hf,pulses)):
            n_exp_hf=puls_hf[0]
            n_exp=puls[0]
            fl_hf=n_exp_hf[3]+n_exp_hf[4]+n_exp_hf[5]
            fl=n_exp[3]+n_exp[4]+n_exp[5]
            flur_hf=np.append(flur_hf,np.sum(fl_hf))
            flur=np.append(flur,np.sum(fl))
        print("\n\nFinally ended!! The next 3 code blocks have the plots")
        return flur,flur_hf,times
    flur_rabi,flur_hf_rabi,times = rabi_para(t,n)   

#### Plots

In [ ]:
def norm(arr):
    return (arr-np.min(arr))/(np.max(arr)-np.min(arr))
def stand(arr):
    return (arr-np.mean(arr))/(np.max(arr)-np.min(arr))
if run_rabi:   
    norm_flur_hf_rabi = norm(stand(flur_hf_rabi))
    norm_flur_rabi = norm(stand(flur_rabi))
    puls_rabi = pulsed_mg(0.4,0.6,10.0,1.1,0.0,protocol_repeats=1,progress_bar="OFF")
    puls_rabi_hf = pulsed_mg_hf(0.4,0.6,10.0,1.1,0.0,protocol_repeats=1,progress_bar="OFF")
    plt.title(f"PL\n Laser:{puls_rabi[4]} $\\mu$s, Delay:{puls_rabi[5]} $\\mu$s, Pulse duration:{puls_rabi[6]} $\\mu$s",fontsize=20)
    plt.plot(times,norm_flur_hf_rabi,label="HF")
    plt.plot(times,norm_flur_rabi,label="No-HF")
    plt.xlabel("MW active time ($\\mu$s)",fontsize=20)
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.minorticks_on()

In [ ]:
if run_rabi: 
    plt.xlabel("MW active time ($\\mu$s)")
    plt.scatter(times,norm_flur_hf_rabi)
    def exp(x, a, b, c):
        return c - a * np.exp(- b * x)
    def fit(x, a, b, c, d, e):
        return e - a * np.exp(- b * x) * np.cos(c * x + d)
    fitt=latexify.get_latex(fit)
    fitt=fitt.replace("\\mathopen{}","")
    fitt=fitt.replace("\\mathclose{}","")
    popt_hf, pcov_hf = curve_fit(fit, times, norm_flur_hf_rabi,p0=[-0.5,0.1,15.6,-0.001,0.01])
    plt.plot(times, fit(times, *popt_hf), 'black', ls="--",
             label='fit: a=%5.3f, b=%5.3f, c=%5.3f,\n       d=%5.3f, e=%5.3f' % tuple(popt_hf))
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.title(f"PL \\w HF\n Laser:{puls_rabi_hf[4]} $\\mu$s, Delay:{puls_rabi_hf[5]} $\\mu$s, Pulse duration:{puls_hf   [6]} $\\mu$s\n${fitt}$",fontsize=20)
    times_cont=np.extract(puls_rabi_hf[1]>puls_rabi_hf[-2],puls_rabi_hf[1])
    r=np.full_like(times_cont,times_cont[0]-t/n)
    times_cont=np.subtract(times_cont,r)
    ydata=puls_hf[0][0][-len(times_cont):]
    r=np.full_like(ydata,0.1345)
    ydata=np.add(ydata,r)
    plt.plot(times_cont,norm(stand(ydata)),label="$n_1$")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.minorticks_on()

In [ ]:
if run_rabi:
    plt.xlabel("MW active time ($\\mu$s)")
    plt.scatter(times,norm_flur_rabi)
    def exp(x, a, b, c):
        return c - a * np.exp(- b * x)
    def fit(x, a, b, c, d, e):
        return e - a*np.exp(- b * x) *np.cos(c * x + d)
    fitt=latexify.get_latex(fit)
    fitt=fitt.replace("\\mathopen{}","")
    fitt=fitt.replace("\\mathclose{}","")
    popt, pcov = curve_fit(fit, times, norm_flur_rabi,p0=[-0.5,0.1,15.6,-0.001,0.01])
    plt.plot(times, fit(times, *popt), 'black', ls="--",
             label='fit: a=%5.3f, b=%5.3f, c=%5.3f,\n       d=%5.3f, e=%5.3f' % tuple(popt))
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.title(f"PL \\wo HF\n Laser:{puls_rabi[4]} $\\mu$s, Delay:{puls_rabi[5]} $\\mu$s, Pulse duration:{puls_rabi[6]}   $\\mu$s\n${fitt}$",fontsize=20)
    times_cont=np.extract(puls_rabi[1]>puls_rabi[-2],puls_rabi[1])
    r=np.full_like(times_cont,times_cont[0]-t/n)
    times_cont=np.subtract(times_cont,r)
    ydata=puls[0][0][-len(times_cont):]
    r=np.full_like(ydata,0.1345)
    ydata=np.add(ydata,r)
    plt.plot(times_cont,norm(stand(ydata)),label="$n_1$")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.minorticks_on()

# Model with Zero-Field Splitting Hamiltonian

## Introduction
...

## Mathematical Formulation
$$
H_{ZFS,j} = D_j S_{z,j}^2 \\
H_i = H_{ZFS,j} + \mu_e \bold{B} \cdot \hat{S_j} \\
H_{int,j} = \sqrt{2} \Omega_r \left(S_{x,j} cos(wt)cos(\phi)+S_{y,j} cos(wt)sin(\phi)\right)\\
j=\{gs,es\}, \ \Omega_r=\mu_e\lVert\bold{B_{MW}}\rVert
$$

## Simulation Code

### Hamiltonian, constants and dynamics functions

In [ ]:
# Dimention of the Hilbert space
dim = 7
# Optical transition rates index
k=2
# dephasing times(microseconds)
t1_gs=1e3
t2_gs=2
t1_es=1e3
t2_es=6*1e-3
    
mu_e = 2.8 # (Mhz/G)
B = B0(500.0,0.0,0.0) # (G)
# Constants (MHz)
D_gs = 2870.0
D_es = 1420.0
W_p = 1.9
gamma_gs = 1/t1_gs,1/t2_gs
gamma_es = 1/t1_es,1/t2_es

# Microwave frequency
omega = D_gs-mu_e*B[2]
# Microwave phase
phi = 0.0
# I had to modify this two value to repruduce the results of the paper
# Microwave magnectic field*mu_e 
Om_r=15.7

def H_zfs(b, om_r):
    """
    Calculates the Hamiltonian for a given magnetic field and Rabi frequency.
    
    Parameters:
    b (np.array): Magnetic field vector B0(B_amp,phi_B,theta_B).
    om_r (float): Rabi frequency.
    
    Returns:
    list: A list containing the Hamiltonian Qobj terms.
    """
    H_0 = [D_gs*sz_gs**2+mu_e*np.dot(b,S_gs),
           D_es*sz_es**2+mu_e*np.dot(b,S_es)]
    H_int = [[np.sqrt(2)*om_r*(sx_gs), "cos(w*t)*cos(p)"],
           [np.sqrt(2)*om_r*(sy_gs), "cos(w*t)*sin(p)"],
           [np.sqrt(2)*om_r*(sx_es), "cos(w*t)*cos(p)"],
           [np.sqrt(2)*om_r*(sy_es), "cos(w*t)*sin(p)"]]
    return [*H_0,*H_int]

def L_zfs(w_p,k_index=k_ind):
    """
    Returns the Lindblad operators of the system, including optical transitions based on the given k_index.

    Parameters:
    - w_p (float): Laser pump rate.
    - k_index (int, optional): Index for the optical transition rates. Defaults to k.

    Returns:
    - c_ops (list): List of Lindblad operators Qobj.
    """
    #optical transitions
    K_s=[[66.0,0.0,57.0,1.0,0.7],
         [77.0,0.0,30.0,3.3,0.0],
         [62.7,12.97,80.0,3.45,1.08],
         [63.2,10.8,60.7,0.8,0.4],
         [67.4,9.9,96.6,4.83,1.055]]
    k41 = K_s[k_index][0]
    k52 = K_s[k_index][0]
    k63 = K_s[k_index][0]
    k57 = K_s[k_index][2]
    k67 = K_s[k_index][2]
    k47 = K_s[k_index][1]
    k71 = K_s[k_index][3]
    k72 = K_s[k_index][4]
    k73 = K_s[k_index][4]
    
    c_ops = []

    c_ops.append(np.sqrt(w_p) * (excited[1] * ground[1].dag()))  # n1 to n4 #type: ignore
    c_ops.append(np.sqrt(w_p) * (excited[2] * ground[2].dag()))  # n2 to n5 #type: ignore
    c_ops.append(np.sqrt(w_p) * (excited[0] * ground[0].dag())) # n3 to n6 #type: ignore

    c_ops.append(np.sqrt(k41) * (ground[1] * excited[1].dag()))  # n4 to n1 #type: ignore
    c_ops.append(np.sqrt(k71) * (ground[1] * isc.dag()))  # n7 to n1 #type: ignore

    c_ops.append(np.sqrt(k52) * (ground[2] * excited[2].dag()))  # n5 to n2  #type: ignore
    c_ops.append(np.sqrt(k72) * (ground[2] * isc.dag()))  # n7 to n2 #type: ignore

    c_ops.append(np.sqrt(k63) * (ground[0] * excited[0].dag()))  # n6 to n3 #type: ignore
    c_ops.append(np.sqrt(k73) * (ground[0] * isc.dag()))  # n7 to n3 #type: ignore

    c_ops.append(np.sqrt(k47) * (isc * excited[1].dag()))  # n4 to n7 #type: ignore
    c_ops.append(np.sqrt(k57) * (isc * excited[2].dag()))  # n5 to n7 #type: ignore
    c_ops.append(np.sqrt(k67) * (isc * excited[0].dag()))  # n6 to n7 #type: ignore
    # Add collapse operators for decoherence
    c_ops.append(np.sqrt(gamma_gs[1]) * sz_gs)
    c_ops.append(np.sqrt(gamma_gs[0]/2) * (sm_gs))
    c_ops.append(np.sqrt(gamma_gs[0]/2) * (sp_gs))
    c_ops.append(np.sqrt(gamma_es[1]) * sz_es)
    c_ops.append(np.sqrt(gamma_es[0]/2) * (sm_es))
    c_ops.append(np.sqrt(gamma_es[0]/2) * (sp_es))
    
    return c_ops


def dynamics_zfs(
    dt,
    init_state,
    b=None,
    om=None,
    p=None,
    om_r=None,
    w_p=None,
    k_index=k_ind,
    exp_ops=None,
    ti=0.0,    
    mode="Free",
    progress_bar="ON",
    i=0,
):
    """
    Perform dynamics_zfs simulation based on the given parameters, including optical transition rates index.
    Where, when using k_index=2 -> K_s[k_index]=[62.7,12.97,80.0,3.45,1.08], and the full K_s list is:
        K_s=[[66.0,0.0,57.0,1.0,0.7],
             [77.0,0.0,30.0,3.3,0.0],
             [62.7,12.97,80.0,3.45,1.08],
             [63.2,10.8,60.7,0.8,0.4],
             [67.4,9.9,96.6,4.83,1.055]]
    Parameters:
    - dt (float): Time step for the calculations.
    - init_state: Initial state for the simulation.
    - b (np.array, optional): Magnetic field vector. Defaults to B0(B_amp,phi_B,theta_B).
    - om (float, optional): Angular frequency of the system. Defaults to omega.
    - om_r (float, optional): Angular frequency for MW-ON evolution. Defaults to Om_r.
    - w_p (float, optional): Frequency for laser-ON evolution. Defaults to W_p.
    - k_index=k_index (int, optional): Index for the optical transition rates. Defaults to k.
    - exp_ops (list, optional): List of expectation operators. Defaults to 
        [n1, n2, n3, n4, n5, n6, n7, nc, 
        sx_gs, sy_gs, sz_gs, 
        sx_es, sy_es, sz_es].
    - ti (float, optional): Initial time for the simulation. Defaults to 0.0.
    - mode (str, optional): Mode of the simulation. Can be "Free", "MW", "Laser", or "Laser-MW". Defaults to "Free".
    - progress_bar (str, optional): Progress bar option. Can be "ON" or "OFF". Defaults to "ON".
    - i (int, optional): Iteration number. Defaults to 0.

    Returns:
    - tf (float): Final time of the simulation.
    - result: Result of the simulation.
    """
    # Default values
    if b is None: b = B
    if om is None: om = omega
    if p is None: p = phi
    if om_r is None: om_r = Om_r
    if w_p is None: w_p = W_p
    if exp_ops is None: exp_ops = [
        n1, n2, n3, n4, n5, n6, n7, nc, 
        sx_gs, sy_gs, sz_gs, 
        sx_es, sy_es, sz_es
    ]

    # Arguments for the Hamiltonian
    args = {"w": om, "p": p}
    
    # Define the time resolution
    t_bins = 1000 if dt <= 5 else 5000
    
    tf = ti + dt

    # Define collapse operators and Hamiltonian based on mode
    match mode:
        case "Free":
            c_ops = L_zfs(0.0, k_index=k_index)
            H = H_zfs(b, 0.0)
        case "MW":
            c_ops = L_zfs(0.0, k_index=k_index)
            H = H_zfs(b, om_r)
        case "Laser":
            c_ops = L_zfs(w_p, k_index=k_index)
            H = H_zfs(b, 0.0)
        case "Laser-MW":
            c_ops = L_zfs(w_p, k_index=k_index)
            H = H_zfs(b, om_r)
        case _:
            raise ValueError('mode must be one of "Free", "MW", "Laser", or "Laser-MW"')
    
    
    # Call the master equation solver
    match progress_bar:
        case "OFF":
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                e_ops=exp_ops,
                args=args,
                options={"store_states": True},
            )
        case "ON":
            print(f"{mode} {int(i + 1)} \n ti | tf \n {int(ti)} | {int(tf)}")
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                e_ops=exp_ops,
                args=args,
                options={"store_states": True, "progress_bar": "tqdm"},
            )
        case _:
            raise ValueError('progress_bar must be "ON" or "OFF"')
    
    return tf, result

### Continuous laser dynamics function

In [ ]:
def continuous_zfs(laser_time, mw_time, free_time, b=B, om_r=Om_r, om=omega, w_p=W_p,protocol_repeats=1, progress_bar="ON"):
    """
    Perform continuous evolution of a quantum system under a specific protocol.

    Args:
        b (float): Magnetic field strength.
        om_r (float): Rabi frequency of the resonant microwave field.
        om (float): Rabi frequency of the off-resonant microwave field.
        laser_time (float): Duration of the laser-on period.
        mw_time (float): Duration of the microwave-on period.
        free_time (float): Duration of the free evolution period.
        protocol_repeats (int, optional): Number of times to repeat the protocol. Defaults to 1.
        progress_bar (str, optional): Progress bar display option. Defaults to "ON".

    Returns:
        tuple: A tuple containing the following arrays:
            - n_exp (numpy.ndarray): Expectation values of the selected states.
            - times (numpy.ndarray): Times at which the expectation values are calculated.
            - tis (numpy.ndarray): Times at which the laser and microwave fields are activated.
            - tfs (numpy.ndarray): Times at which the laser and microwave fields are deactivated.
    """
    # Initial state where the ground state is evenly populated
    init_state = 1 / 3 * (n1 + n2 + n3)

    # Array to collect the result Class
    results = np.array([])

    # Initial time of the evolution
    ti = 0.0

    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis = []
    tfs = []

    # Set the number of times to repeat the protocol
    n = protocol_repeats
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = laser_time
    dt_free = free_time
    dt_mw = mw_time
    print("Starting continuous calculations, this may take a while...")
    # This loop is defined to reproduce the same protocol as a pulsed protocol, but with continuous laser
    for i in tqdm(range(n)):
        tis.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_zfs(dt_laser,
                                  init_state,
                                  b=b,
                                  om_r=om_r,
                                  om=om,
                                  w_p=w_p,
                                  ti=ti,
                                  mode="Laser",
                                  progress_bar=progress_bar,
                                  i=i) # type:ignore
        # Store the Result class for when the Laser is on
        results = np.append(results, result) # type: ignore
        # Save the initial state for the next part of the protocol
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_zfs(dt_free,
                                  init_state,
                                  b=b,
                                  om_r=om_r,
                                  om=om,
                                  w_p=w_p,
                                  ti=ti,
                                  mode="Free",
                                  progress_bar=progress_bar,
                                  i=i) # type:ignore
        # Store the Result class for MW on evolution
        results = np.append(results, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_zfs(dt_mw,
                                  init_state,
                                  b=b,
                                  om_r=om_r,
                                  om=om,
                                  w_p=w_p,
                                  ti=ti,
                                  mode="MW",
                                  progress_bar=progress_bar,
                                  i=i) # type:ignore
        # Store the Result class for MW on evolution
        results = np.append(results, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
    # Gather the expectation values from the results array
    for j in range(len(results[0].expect)):
        nn_exp = np.array([])
        for i in range(len(results)):
            nn_exp = np.append(nn_exp, results[i].expect[j])
        if j == 0:
            n_exp_zfs = np.array([nn_exp])
        else:
            n_exp_zfs = np.append(n_exp_zfs, [nn_exp], axis=0) # type: ignore
    if len(n_exp_zfs)>8:
        s_exp_zfs=n_exp_zfs[len(n_exp_zfs)-6:]
        n_exp_zfs=n_exp_zfs[:len(n_exp_zfs)-6]
    # Gather the times for the plots
    times = np.array([])
    for i in range(len(results)):
        times = np.append(times, results[i].times)
    print("Calculations are finished!! param order(Expect,times,tis,tfs,results,s_exp)")
    return n_exp_zfs, times, tis, tfs, results,s_exp_zfs

### Pulsed laser dynamics function

In [ ]:
def pulsed_zfs(active_laser_time, delay, total_pulse_time, 
                mw_time, free_time,b=B, om_r=Om_r, om=omega, w_p=W_p,
                protocol_repeats=1,progress_bar="ON"):
    """
    Simulates the evolution of a quantum system under a given protocol with pulsed laser.

    Args:
        active_laser_time (float): The duration of the laser activation in each iteration.
        delay (float): The delay time between laser activations.
        total_pulse_time (float): The total time for the pulsed laser.
        mw_time (float): The duration of the microwave activation in each iteration.
        free_time (float): The duration of the free evolution time in each iteration.
        protocol_repeats (int, optional): The number of times to repeat the protocol. Defaults to 1.

    Returns:
        tuple: A tuple containing the following elements:
            - n_p_exp (numpy.ndarray): An array of expectation values for different states.
            - times_p (numpy.ndarray): An array of times for the plots.
            - tis (list): A list of activation times for laser and microwave.
            - tfs (list): A list of deactivation times for laser and microwave.
            - active_laser_time (float): The duration of the laser activation.
            - delay (float): The delay time between laser activations.
            - total_pulse_time (float): The total time for the pulsed laser.
            - results_p (numpy.ndarray): An array of the Result class object for the time evolution.
    """
    # Initial state where the ground state is evenly populated
    init_state = 1/3 * (n1 + n2 + n3)

    # Array to collect the result Class
    results_p = np.array([])

    # Initial time of the evolution
    ti = 0.0

    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis = []
    tfs = []

    # Set the number of times to repeat the protocol
    m = protocol_repeats
    # Set the time intervals for each part of the evolution (with pulse laser, free evolution of pulsed laser , with microwaves and free evolution)
    dt_laser = active_laser_time
    dt_free_pulse = delay
    t_laser_pulse = total_pulse_time
    dt_mw = mw_time
    dt_free = free_time
    # Set the number of times to repeat the pulsed laser
    n = int(t_laser_pulse // (dt_laser + dt_free_pulse))
    print("Starting pulse laser calculations, this may take a while...")
    # This loop must be defined in a way to reproduce the protocol choosen
    for j in tqdm(range(m),desc="Protocol"):
        tis.append(ti)
        for i in tqdm(range(n),desc="Pulse",leave=False):
            # Here Laser is ON
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_zfs(dt_laser,
                                      init_state,
                                      b=b,
                                      om_r=om_r,
                                      om=om,
                                      w_p=w_p,
                                      ti=ti,
                                      mode="Laser",
                                      progress_bar="OFF",
                                      i=i) # type:ignore 
            # Store the Result class for when the Laser is on
            results_p = np.append(results_p, result) # type: ignore
            # Make initial time to be the end time of the last part of the protocol
            ti = tf
            # Make the last state  of this evolution step be the initial states of  the next iteration
            init_state = result.states[-1]
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_zfs(dt_free_pulse,
                                      init_state,
                                      b=b,
                                      om_r=om_r,
                                      om=om,
                                      w_p=w_p,
                                      ti=ti,
                                      mode="Free",
                                      progress_bar="OFF",
                                      i=i) # type:ignore 
            # Store the Result class for metastable state depletion evolution
            results_p = np.append(results_p, result) # type: ignore
            # Save the initial state for the next part of the protocol
            init_state = result.states[-1]
            # Make the initial time the end time of the previous part of the procotol
            ti = tf
        tfs.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_zfs(dt_free,
                                  init_state,
                                  b=b,
                                  om_r=om_r,
                                  om=om,
                                  w_p=w_p,
                                  ti=ti,
                                  mode="Free",
                                  progress_bar=progress_bar,
                                  i=j) # type:ignore 
        # Store the Result class for MW on evolution
        results_p = np.append(results_p, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the  next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_zfs(dt_mw,
                                  init_state,
                                  b=b,
                                  om_r=om_r,
                                  om=om,
                                  w_p=w_p,
                                  ti=ti,
                                  mode="MW",
                                  progress_bar=progress_bar,
                                  i=j) # type:ignore 
        # Store the Result class for MW on evolution
        results_p = np.append(results_p, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
    # Gather the expectation values from the results array
    for j in range(len(results_p[0].expect)):
        nn_exp = np.array([])
        for i in range(len(results_p)):
            nn_exp = np.append(nn_exp, results_p[i].expect[j])
        if j == 0:
            n_p_exp_zfs = np.array([nn_exp])
        else:
            n_p_exp_zfs = np.append(n_p_exp_zfs, [nn_exp], axis=0) # type: ignore
    if len(n_p_exp_zfs)>8:
        s_p_exp_zfs=n_p_exp_zfs[len(n_p_exp_zfs)-6:]
        n_p_exp_zfs=n_p_exp_zfs[:len(n_p_exp_zfs)-6]
    # Gather the times for the plots
    times_p = np.array([])
    for i in range(len(results_p)):
        times_p = np.append(times_p, results_p[i].times)
    print(
        "Calculations are finished!! param order(Expect,times,tis,tfs,active_laser_time,delay,total_pulse_time,results_p,s_p_exp)"
    )
    return (
        n_p_exp_zfs,
        times_p,
        tis,
        tfs,
        active_laser_time,
        delay,
        total_pulse_time,
        results_p,
        s_p_exp_zfs,
    )

## Results and Discussion
...

In [ ]:
cont_zfs=continuous_zfs(10.0,1.0,1.0,progress_bar="OFF")

In [ ]:
plot_continuous(cont_zfs[0],cont_zfs[1],cont_zfs[2],cont_zfs[3])

### Plots

In [ ]:
puls_zfs=pulsed_zfs(0.4,0.6,10.0,1.0,1.0,protocol_repeats=1,progress_bar="OFF")

In [ ]:
plot_pulsed(
    puls_zfs[0],
    puls_zfs[1],
    puls_zfs[2],
    puls_zfs[3],
    puls_zfs[4],
    puls_zfs[5],
    puls_zfs[6],
)

In [ ]:
plot_pulsed_v_cont(
    puls_zfs[0],
    puls_zfs[1],
    puls_zfs[2],
    puls_zfs[3],
    puls_zfs[4],
    puls_zfs[5],
    puls_zfs[6],
    cont_zfs[0],
    cont_zfs[1])

# Comprehensive Model: Zero-Field Splitting and Hyperfine Interaction

## Introduction
...

## Mathematical Formulation
$$
H_{hf,j} = A^j_\parallel S_{z,j} I_z + A^j_\perp (S_{x,j} I_x + S_{y,j} I_y) \\	
H_j = \left(H_{ZFS,j} +\mu_e \bold{B} \cdot \hat{S_j}\right)\otimes\mathbb{I}_N + \mathbb{I}_{NV} \otimes \left(\mu_n \bold{B} \cdot \hat{I}\right) + H_{hf}\\
H_{int,j} = \sqrt{2}\Omega_r \left(S_{x,j} cos(wt)cos(\phi)+S_{y,j} cos(wt)sin(\phi)\right)\otimes\mathbb{I}_N \\
H_{int,N} = \mathbb{I}_{NV} \otimes 2 \mu_n \lVert\bold{B_{MW}}\rVert \left(I_x cos(wt)cos(\phi)+I_y cos(wt)sin(\phi)\right) \\
N=\{^{14}N \ \text{or}  \ ^{15}N\} \\
j=\{gs,es\}, \ \Omega_r=\mu_e\lVert\bold{B_{MW}}\rVert
$$

## Simulation Code

### Hamiltonian, constants and dynamics functions

In [ ]:
# Dimention of the Hilbert space
dim = 7

t1_n=1e5
t2_n=1e3
# Constants (MHz)
gamma_n=1/t1_n,1/t2_n
mu_n = 431.7*1e-6 # (MHz/G)

a_gs=3.03,3.65
a_es=-57.8,-39.2

def H_zfs_hf(b, om_r):
    """
    Calculate the Hamiltonian for a given magnetic field and resonant frequency.
    
    Parameters:
    b (numpy.ndarray): Magnetic field vector [bx,by,bz].
    om_r: Resonant frequency.
    
    Returns:
    list: List of Hamiltonian terms.
    """
    
    H_0 = [(D_gs*sz_gs**2+mu_e*np.dot(b,S_gs))&IdN15,
           (D_es*sz_es**2+mu_e*np.dot(b,S_es))&IdN15,
           a_gs[0]*(sz_gs&sz_n) + a_gs[1]*((sx_gs&sx_n) + (sy_gs&sy_n)),
           a_es[0]*(sz_es&sz_n) + a_es[1]*((sx_es&sx_n) + (sy_es&sy_n))]
    H_n = [IdNV&(mu_n*np.dot(b,S_n))]
    H_int=[[(np.sqrt(2)*om_r*(sx_gs))&IdN15, "cos(w*t)*cos(p)"],
           [(np.sqrt(2)*om_r*(sy_gs))&IdN15, "cos(w*t)*sin(p)"],
           [(np.sqrt(2)*om_r*(sx_es))&IdN15, "cos(w*t)*cos(p)"],
           [(np.sqrt(2)*om_r*(sy_es))&IdN15, "cos(w*t)*sin(p)"],
           [IdNV&(2*om_r/mu_e*mu_n*(sx_n)), "cos(w*t)*cos(p)"],
           [IdNV&(2*om_r/mu_e*mu_n*(sy_n)), "cos(w*t)*sin(p)"]]
    return [*H_0,*H_n,*H_int] 

def L_zfs_hf(w_p,k_index=k_ind):
    """
    Returns the Lindblad operators of the system, including optical transitions based on the given k_index.

    Parameters:
    - w_p (float): Laser pump rate.
    - k_index (int, optional): Index for the optical transition rates. Defaults to k.

    Returns:
    - c_ops (list): List of Lindblad operators.
    """
    K_s=[[66.0,0.0,57.0,1.0,0.7],
         [77.0,0.0,30.0,3.3,0.0],
         [62.7,12.97,80.0,3.45,1.08],
         [63.2,10.8,60.7,0.8,0.4],
         [67.4,9.9,96.6,4.83,1.055]]
    k41 = K_s[k_index][0]
    k52 = K_s[k_index][0]
    k63 = K_s[k_index][0]
    k57 = K_s[k_index][2]
    k67 = K_s[k_index][2]
    k47 = K_s[k_index][1]
    k71 = K_s[k_index][3]
    k72 = K_s[k_index][4]
    k73 = K_s[k_index][4]
    
    c_ops = []

    c_ops.append((np.sqrt(w_p) * (excited[1] * ground[1].dag()))&IdN15)  # n1 to n4 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[2] * ground[2].dag()))&IdN15)  # n2 to n5 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[0] * ground[0].dag()))&IdN15) # n3 to n6 #type: ignore

    c_ops.append((np.sqrt(k41) * (ground[1] * excited[1].dag()))&IdN15)  # n4 to n1 #type: ignore
    c_ops.append((np.sqrt(k71) * (ground[1] * isc.dag()))&IdN15)  # n7 to n1 #type: ignore

    c_ops.append((np.sqrt(k52) * (ground[2] * excited[2].dag()))&IdN15)  # n5 to n2 #type: ignore
    c_ops.append((np.sqrt(k72) * (ground[2] * isc.dag()))&IdN15)  # n7 to n2 #type: ignore

    c_ops.append((np.sqrt(k63) * (ground[0] * excited[0].dag()))&IdN15)  # n6 to n3 #type: ignore
    c_ops.append((np.sqrt(k73) * (ground[0] * isc.dag()))&IdN15)  # n7 to n3 #type: ignore

    c_ops.append((np.sqrt(k47) * (isc * excited[1].dag()))&IdN15)  # n4 to n7 #type: ignore
    c_ops.append((np.sqrt(k57) * (isc * excited[2].dag()))&IdN15)  # n5 to n7 #type: ignore
    c_ops.append((np.sqrt(k67) * (isc * excited[0].dag()))&IdN15)  # n6 to n7 #type: ignore
    return c_ops

def dynamics_zfs_hf(
    dt,
    init_state,
    b=None,
    om_r=None,
    om=None,
    p=None,
    w_p=None,
    k_index=k_ind,
    exp_ops=None,
    ti=0.0,
    mode="Free",
    progress_bar="ON",
    i=0,
):
    """
    Simulate the dynamics of a quantum system under hyperfine interaction using the Hamiltonian and collapse operators.
    including optical transition rates index.
    Where, when using k_index=2 -> K_s[k_index]=[62.7,12.97,80.0,3.45,1.08], and the full K_s list is:
        K_s=[[66.0,0.0,57.0,1.0,0.7],
             [77.0,0.0,30.0,3.3,0.0],
             [62.7,12.97,80.0,3.45,1.08],
             [63.2,10.8,60.7,0.8,0.4],
             [67.4,9.9,96.6,4.83,1.055]]
    Parameters:
    - dt (float): Total simulation time.
    - init_state (qutip.Qobj): Initial quantum state of the system.
    - b (np.array, optional): Magnetic field vector. Defaults to B0(B_amp,phi_B,theta_B).
    - om_r (float, optional): Rabi frequency for microwave interactions. Defaults to Om_r.
    - om (float, optional): Angular frequency of the system. Defaults to omega.
    - p (float, optional): Microwave phase. Defaults to 0.0.
    - w_p (float, optional): Laser frequency. Defaults to W_p.
    - k_index(int, optional): Index for the optical transition rates. Defaults to k.
    - exp_ops (list of qutip.Qobj, optional): List of operators for which to compute expectation values. 
        Defaults to a list of tensor products including different Pauli operators and state projectors.
    - ti (float, optional): Initial time of the simulation. Defaults to 0.0.
    - mode (str, optional): Simulation mode. Can be "Free", "MW", "Laser", or "Laser-MW". Defaults to "Free".
    - progress_bar (str, optional): Option to display a progress bar. Can be "ON" or "OFF". Defaults to "ON".
    - i (int, optional): Counter for the progress bar. Defaults to 0.

    Returns:
    - tf (float): Final time of the simulation.
    - result (qutip.solver.Result): Result object containing the simulation output.
    """
    # Default values
    if b is None: b = B
    if om_r is None: om_r = Om_r
    if om is None: om = omega
    if p is None: p = phi
    if w_p is None: w_p = W_p
    if exp_ops is None: exp_ops = [
        n1&IdN15, n2&IdN15, n3&IdN15,
        n4&IdN15, n5&IdN15, n6&IdN15,
        n7&IdN15, nc&IdN15,
        sx_gs&IdN15, sy_gs&IdN15, sz_gs&IdN15,
        sx_es&IdN15, sy_es&IdN15, sz_es&IdN15,
        IdNV&nit[0] * nit[0].dag(), IdNV&nit[1] * nit[1].dag() #type: ignore
    ]

    # Time resolution based on dt
    t_bins = 1000 if dt <= 5 else 5000
    
    # Define Hamiltonian and collapse operators based on mode
    match mode:
        case "Free":
            H = H_zfs_hf(b, 0.0)
            c_ops = L_zfs_hf(0.0, k_index=k_index)
        case "MW":
            H = H_zfs_hf(b, om_r)
            c_ops = L_zfs_hf(0.0, k_index=k_index)
        case "Laser":
            H = H_zfs_hf(b, 0.0)
            c_ops = L_zfs_hf(w_p, k_index=k_index)
        case "Laser-MW":
            H = H_zfs_hf(b, om_r)
            c_ops = L_zfs_hf(w_p, k_index=k_index)
        case _:
            raise ValueError('mode must be one of "Free", "MW", "Laser", or "Laser-MW"')

    # Add collapse operators for decoherence   
    c_ops.append((np.sqrt(gamma_gs[1]-gamma_gs[0]/2) * sz_gs)&IdN15)
    c_ops.append((np.sqrt(gamma_gs[0]) * (sm_gs))&IdN15)
    c_ops.append((np.sqrt(gamma_es[1]-gamma_es[0]/2) * sz_es)&IdN15)
    c_ops.append((np.sqrt(gamma_es[0]) * (sm_es))&IdN15)
    c_ops.append(IdNV&(np.sqrt(gamma_n[1]-gamma_n[0]/2) * sz_n))
    c_ops.append(IdNV&(np.sqrt(gamma_n[0]) * (sm_n)))

    # Arguments for the Hamiltonian
    args = {"w": om, "p": p}
    
    tf = ti + dt

    # Solve the master equation
    match progress_bar:
        case "OFF":
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                e_ops=exp_ops,
                args=args,
                options={"store_states": True},
            )
        case "ON":
            print(f"{mode} {int(i + 1)} \n ti | tf \n {int(ti)} | {int(tf)}")
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                e_ops=exp_ops,
                args=args,
                options={"store_states": True, "progress_bar": "tqdm"},
            )
        case _:
            raise ValueError('progress_bar must be "ON" or "OFF"')

    return tf, result

### Continuous laser dynamics function

In [ ]:
def continuous_zfs_hf(laser_time, mw_time, free_time,b=B, om_r=Om_r, om=omega, w_p=W_p, 
                      protocol_repeats=1,progress_bar="ON"):
    """
    Simulates the evolution of a quantum system under a given protocol with continuos laser.

    Args:
        laser_time (float): The duration of the laser pulse in each iteration.
        mw_time (float): The duration of the microwave pulse in each iteration.
        free_time (float): The duration of the free evolution in each iteration.
        protocol_repeats (int, optional): The number of times to repeat the protocol. Defaults to 1.

    Returns:
        tuple: A tuple containing the following arrays:
            - n_exp (numpy.ndarray): An array of expectation values for different operators.
            - times (numpy.ndarray): An array of time values.
            - tis (numpy.ndarray): An array of initial times for each part of the protocol.
            - tfs (numpy.ndarray): An array of final times for each part of the protocol.
            - results (numpy.ndarray): An array of the Result class object for the time evolution.
    """
    # Initial state where the ground state is evenly populated
    init_state = (1 / 3 * (n1 + n2 + n3))&(1/2 * IdN15)

    # Array to collect the result Class
    results = np.array([])

    # Initial time of the evolution
    ti = 0.0

    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis = []
    tfs = []

    # Set the number of times to repeat the protocol
    n = protocol_repeats
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = laser_time
    dt_free = free_time
    dt_mw = mw_time
    print("Starting continuous calculations, this may take a while...")
    # This loop is defined to reproduce the same protocol as a pulsed protocol, but with continuous laser
    for i in tqdm(range(n)):
        tis.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_zfs_hf(dt_laser,
                                     init_state,
                                     b=b,
                                     om_r=om_r,
                                     om=om,
                                     w_p=w_p,
                                     ti=ti,
                                     mode="Laser",
                                     progress_bar=progress_bar,
                                     i=i) # type:ignore
        # Store the Result class for when the Laser is on
        results = np.append(results, result) # type: ignore
        # Save the initial state for the next part of the protocol
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_zfs_hf(dt_free,
                                     init_state,
                                     b=b,
                                     om_r=om_r,
                                     om=om,
                                     w_p=w_p,
                                     ti=ti,
                                     mode="Free",
                                     progress_bar=progress_bar,
                                     i=i) # type:ignore
        # Store the Result class for MW on evolution
        results = np.append(results, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_zfs_hf(dt_mw,
                                     init_state,
                                     b=b,
                                     om_r=om_r,
                                     om=om,
                                     w_p=w_p,
                                     ti=ti,
                                     mode="MW",
                                     progress_bar=progress_bar,
                                     i=i) # type:ignore
        # Store the Result class for MW on evolution
        results = np.append(results, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
    # Gather the expectation values from the results array
    for j in range(len(results[0].expect)):
        nn_exp = np.array([])
        for i in range(len(results)):
            nn_exp = np.append(nn_exp, results[i].expect[j])
        if j == 0:
            n_exp = np.array([nn_exp])
        else:
            n_exp = np.append(n_exp, [nn_exp], axis=0) # type: ignore
    if len(n_exp)>8:
        s_exp=n_exp[len(n_exp)-8:len(n_exp)-2]
        nit_exp=n_exp[len(n_exp)-2:]
        n_exp=n_exp[:len(n_exp)-8]
    # Gather the times for the plots
    times = np.array([])
    for i in range(len(results)):
        times = np.append(times, results[i].times)
    print("Calculations are finished!! param order(Expect,times,tis,tfs,results,nit_exp,s_exp)")
    return n_exp, times, tis, tfs, results, nit_exp,s_exp

### Pulsed laser dynamics function

In [ ]:
def pulsed_zfs_hf(active_laser_time, delay, total_pulse_time, 
                  mw_time, free_time,b=B, om_r=Om_r, om=omega, w_p=W_p,
                  protocol_repeats=1,progress_bar="ON"):
    """
    Simulates the evolution of a quantum system under a given protocol with pulsed laser.

    Args:
        active_laser_time (float): The duration of the laser activation in each iteration.
        delay (float): The delay time between laser activations.
        total_pulse_time (float): The total time for the pulsed laser.
        mw_time (float): The duration of the microwave activation in each iteration.
        free_time (float): The duration of the free evolution time in each iteration.
        protocol_repeats (int, optional): The number of times to repeat the protocol. Defaults to 1.

    Returns:
        tuple: A tuple containing the following elements:
            - n_p_exp (numpy.ndarray): An array of expectation values for different states.
            - times_p (numpy.ndarray): An array of times for the plots.
            - tis (list): A list of activation times for laser and microwave.
            - tfs (list): A list of deactivation times for laser and microwave.
            - active_laser_time (float): The duration of the laser activation.
            - delay (float): The delay time between laser activations.
            - total_pulse_time (float): The total time for the pulsed laser.
            - results_p (numpy.ndarray): An array of the Result class object for the time evolution.
    """
    # Initial state where the ground state is evenly populated
    init_state = (1/3*(n1+n2+n3))&(1/2*IdN15)

    # Array to collect the result Class
    results_p = np.array([])

    # Initial time of the evolution
    ti = 0.0

    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis = []
    tfs = []

    # Set the number of times to repeat the protocol
    m = protocol_repeats
    # Set the time intervals for each part of the evolution (with pulse laser, free evolution of pulsed laser , with microwaves and free evolution)
    dt_laser = active_laser_time
    dt_free_pulse = delay
    t_laser_pulse = total_pulse_time
    dt_mw = mw_time
    dt_free = free_time
    # Set the number of times to repeat the pulsed laser
    n = int(t_laser_pulse // (dt_laser + dt_free_pulse))
    print("Starting pulse laser calculations, this may take a while...")
    # This loop must be defined in a way to reproduce the protocol choosen
    for j in tqdm(range(m),desc="Protocol"):
        tis.append(ti)
        for i in tqdm(range(n),desc="Pulse",leave=False):
            # Here Laser is ON
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_zfs_hf(dt_laser,
                                        init_state,
                                        b=b,
                                        om_r=om_r,
                                        om=om,
                                        w_p=w_p,
                                        ti=ti,
                                        mode="Laser",
                                        progress_bar="OFF",
                                        i=i) # type:ignore
            # Store the Result class for when the Laser is on
            results_p = np.append(results_p, result) # type: ignore
            # Make initial time to be the end time of the last part of the protocol
            ti = tf
            # Make the last state  of this evolution step be the initial states of  the next iteration
            init_state = result.states[-1]
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_zfs_hf(dt_free_pulse,
                                              init_state,
                                              b=b,
                                              om_r=om_r,
                                              om=om,
                                              w_p=w_p,
                                              ti=ti,
                                              mode="Free",
                                              progress_bar="OFF",
                                              i=i) # type:ignore
            # Store the Result class for metastable state depletion evolution
            results_p = np.append(results_p, result) # type: ignore
            # Save the initial state for the next part of the protocol
            init_state = result.states[-1]
            # Make the initial time the end time of the previous part of the procotol
            ti = tf
        tfs.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_zfs_hf(dt_free,
                                     init_state,
                                     b=b,
                                     om_r=om_r,
                                     om=om,
                                     w_p=w_p,
                                     ti=ti,
                                     mode="Free",
                                     progress_bar=progress_bar,
                                     i=j) # type:ignore
        # Store the Result class for MW on evolution
        results_p = np.append(results_p, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the  next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_zfs_hf(dt_mw,
                                     init_state,
                                     b=b,
                                     om_r=om_r,
                                     om=om,
                                     w_p=w_p,
                                     ti=ti,
                                     mode="MW",
                                     progress_bar=progress_bar,
                                     i=j) # type:ignore
        # Store the Result class for MW on evolution
        results_p = np.append(results_p, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
    # Gather the expectation values from the results array
    for j in range(len(results_p[0].expect)):
        nn_exp = np.array([])
        for i in range(len(results_p)):
            nn_exp = np.append(nn_exp, results_p[i].expect[j])
        if j == 0:
            n_p_exp = np.array([nn_exp])
        else:
            n_p_exp = np.append(n_p_exp, [nn_exp], axis=0) # type: ignore
    if len(n_p_exp)>8:
        s_p_exp=n_p_exp[len(n_p_exp)-8:len(n_p_exp)-2]
        nit_p_exp=n_p_exp[len(n_p_exp)-2:]
        n_p_exp=n_p_exp[:len(n_p_exp)-8]
    # Gather the times for the plots
    times_p = np.array([])
    for i in range(len(results_p)):
        times_p = np.append(times_p, results_p[i].times)
    print(
        "Calculations are finished!! param order(Expect,times,tis,tfs,active_laser_time,delay,total_pulse_time,results_p,Nit_exp,S_exp)"
    )
    return (
        n_p_exp,
        times_p,
        tis,
        tfs,
        active_laser_time,
        delay,
        total_pulse_time,
        results_p,
        nit_p_exp,
        s_p_exp
    )

## Results and Discussion
...

In [ ]:
cont_zfs_hf=continuous_zfs_hf(10.0,1.0,1.0)

In [ ]:
plot_continuous(cont_zfs_hf[0],cont_zfs_hf[1],cont_zfs_hf[2],cont_zfs_hf[3])

In [ ]:
plot_cont_hf_vs_cont(cont_zfs_hf[0],cont_zfs_hf[1],cont_zfs_hf[2],cont_zfs_hf[3],cont_zfs[0],cont_zfs[1],cont_zfs[2],cont_zfs[3])

In [ ]:
puls_zfs_hf=pulsed_zfs_hf(0.4,0.6,10.0,1.0,1.0,protocol_repeats=1,progress_bar="OFF")

In [ ]:
plot_pulsed(
    puls_zfs_hf[0],
    puls_zfs_hf[1],
    puls_zfs_hf[2],
    puls_zfs_hf[3],
    puls_zfs_hf[4],
    puls_zfs_hf[5],
    puls_zfs_hf[6],
)

In [ ]:
plot_pulsed_v_cont(
    puls_zfs_hf[0],
    puls_zfs_hf[1],
    puls_zfs_hf[2],
    puls_zfs_hf[3],
    puls_zfs_hf[4],
    puls_zfs_hf[5],
    puls_zfs_hf[6],
    cont_zfs_hf[0],
    cont_zfs_hf[1])

In [ ]:
plot_pulsed_hf_v_pulsed(puls_zfs_hf[0],
                        puls_zfs_hf[1],
                        puls_zfs_hf[2],
                        puls_zfs_hf[3],
                        puls_zfs_hf[4],
                        puls_zfs_hf[5],
                        puls_zfs_hf[6],
                        puls_zfs[0],
                        puls_zfs[1],
                        puls_zfs[2],
                        puls_zfs[3],
                        puls_zfs[4],
                        puls_zfs[5],
                        puls_zfs[6])

# Protocols

## Continous Wave Optically Detected Microwave Resonance (CW-ODMR)

In [ ]:
def cwodmr(t, ws, model, init_state, 
           b=B, om_r=Om_r, w_p=W_p):
    """
    Calculate the continuous wave optical double resonance (CWODMR) dynamics_zfs.
    
    Parameters:
    - t (float): Time parameter.
    - ws (numpy.ndarray): Array of angular frequencies.
    - model (function): Function that defines the dynamics_zfs of the system.
    - init_state (object): Initial state of the system.
    - b (float): Parameter b (default: B).
    - om_r (float): Rabi frequency (default: Om_r).
    - w_p (float): Angular frequency (default: W_p).
    
    Returns:
    - ws (numpy.ndarray): Array of angular frequencies.
    - flur (numpy.ndarray): Array of PL values.
    - times (numpy.ndarray): Array of time values.
    - n_exp (numpy.ndarray): Array of expectation values.
    - results (numpy.ndarray): Array of simulation results.
    """
    #continuous MW and laser appplied together
    results = np.array([])
    flur = np.array([])
    for w in tqdm(ws):
        # Run the dynamics_zfs based on the mode choosen
        _, result = model(t, 
                           init_state,
                           b=b,
                           om_r=om_r,
                           om=w,
                           w_p=w_p,
                           ti=0.0, 
                           mode="Laser-MW", 
                           progress_bar="OFF") # type:ignore
        # Gather the expectation values from the results array
        results = np.append(results,result)
        fl=np.array([])
        fl=np.add(result.expect[3],result.expect[4])
        fl=np.add(fl,result.expect[5])
        flur=np.append(flur,np.sum(fl))
    for j in range(len(results[0].expect)):
        nn_exp = np.array([])
        for i in range(len(results)):
            nn_exp = np.append(nn_exp, results[i].expect[j])
        if j == 0:
            n_exp = np.array([nn_exp])
        else:
            n_exp = np.append(n_exp, [nn_exp], axis=0)
    # Gather the times for the plots
    times = np.array([])
    for i in range(len(results)):
        times = np.append(times, results[i].times)
    return ws,flur,times,n_exp,results

### Contrast for CW-ODMR

In [ ]:
def cont_cwodmr(t, ws, model, init_state, 
                b=B, om_r=Om_r, w_p=W_p):
    """
    Calculate the contrast of continuous wave optical detection of magnetic resonance (CW-ODMR).
    
    Parameters:
    - t (float): The duration of the dynamics_zfs.
    - ws (numpy.ndarray): An array of microwave frequencies.
    - model (function): The function that defines the dynamics_zfs of the system.
    - init_state (object): The initial state of the system.
    - b (float): The magnetic field strength. Default is B.
    - om_r (float): The Rabi frequency of the microwave field. Default is Om_r.
    - w_p (float): The pump frequency. Default is W_p.
    
    Returns:
    - ws (numpy.ndarray): An array of microwave frequencies.
    - flur (numpy.ndarray): An array of PL values.
    - times (numpy.ndarray): An array of times for the plots.
    - n_exp (numpy.ndarray): An array of expectation values for each operator.
    - results (numpy.ndarray): An array of simulation results.
    """
    #continuous MW and laser appplied together
    results = np.array([])
    flur = np.array([])
    for w in tqdm(ws):
        # Run the dynamics_zfs based on the mode choosen
        _, result = model(t, 
                           init_state,
                           b=b,
                           om_r=om_r,
                           om=w,
                           w_p=w_p,
                           ti=0.0, 
                           mode="Laser", 
                           progress_bar="OFF") # type:ignore
        # Gather the expectation values from the results array
        results = np.append(results,result)
        fl=np.array([])
        fl=np.add(result.expect[3],result.expect[4])
        fl=np.add(fl,result.expect[5])
        flur=np.append(flur,np.sum(fl))
    for j in range(len(results[0].expect)):
        nn_exp = np.array([])
        for i in range(len(results)):
            nn_exp = np.append(nn_exp, results[i].expect[j])
        if j == 0:
            n_exp = np.array([nn_exp])
        else:
            n_exp = np.append(n_exp, [nn_exp], axis=0)
    # Gather the times for the plots
    times = np.array([])
    for i in range(len(results)):
        times = np.append(times, results[i].times)
    return ws,flur,times,n_exp,results

### Run CW-ODMR

In [ ]:
if run_odmr:
    if b_odmr<600:
        n,m,s=2/5,25,10
        eps=n*b_odmr
    else:
        n,m,s=1/5,50,20
        eps=n*b_odmr
    wslhfl=np.linspace(D_gs-mu_e*b_odmr-mu_e*b_odmr*n,D_gs-mu_e*b_odmr-1.5*a_gs[0],s)
    wslhf=np.linspace(D_gs-mu_e*b_odmr-1.5*a_gs[0],D_gs-mu_e*b_odmr+1.5*a_gs[0],m)
    wslhfh=np.linspace(D_gs-mu_e*b_odmr+1.5*a_gs[0],D_gs-mu_e*b_odmr+mu_e*b_odmr*n,s)
    wsm=np.linspace(D_gs-mu_e*b_odmr+mu_e*b_odmr*n+eps,D_gs+mu_e*b_odmr-mu_e*b_odmr*n-eps,2*s)
    wshhfl=np.linspace(D_gs+mu_e*b_odmr-mu_e*b_odmr*n,D_gs+mu_e*b_odmr-1.5*a_gs[0],s)
    wshhf=np.linspace(D_gs+mu_e*b_odmr-1.5*a_gs[0],D_gs+mu_e*b_odmr+1.5*a_gs[0],m)
    wshhfh=np.linspace(D_gs+mu_e*b_odmr+1.5*a_gs[0],D_gs+mu_e*b_odmr+mu_e*b_odmr*n,s)
    ws=np.unique(np.hstack((wslhfl,wslhf,wslhfh,wsm,wshhfl,wshhf,wshhfh)))
    ws_codm,flur_codm,ts_codm,ns_codm,_=cwodmr(10.0,ws,dynamics_zfs,
                                               1/3*(n1+n2+n3),b=B0(b_odmr,0.0,0.0))
    ws_hf_codm,flur_hf_codm,ts_hf_codm,ns_hf_codm,_=cwodmr(10.0,ws,dynamics_zfs_hf,
                                                           (1/3*(n1+n2+n3))&(1/2*IdN15),b=B0(b_odmr,0.0,0.0))
w_p_odmr=1.5*1.9
if not run_odmr:
    if b_odmr<600:
        n,m,s=2/5,25,10
        eps=n*b_odmr
    else:
        n,m,s=1/5,25,20
        eps=n*b_odmr
    flur_codm,ws_codm =(
        np.load(f'NumpyArrays\\flur_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_W_{int(w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_W_{int(w_p_odmr)}.npy')
    )
    flur_doh_hf_codm,ws_doh_hf_codm =(
        np.load(f'NumpyArrays\\flur_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_W_{int(w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_W_{int(w_p_odmr)}.npy')
    )
    flur_dua_hf_codm,ws_dua_hf_codm =(
        np.load(f'NumpyArrays\\flur_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_W_{int(w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_W_{int(w_p_odmr)}.npy')
    )
    flur_podm,ws_podm =(
        np.load(f'NumpyArrays\\flur_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_W_{int(w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_W_{int(w_p_odmr)}.npy')
    )
    flur_doh_hf_podm,ws_doh_hf_podm =(
        np.load(f'NumpyArrays\\flur_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_W_{int(w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_W_{int(w_p_odmr)}.npy')
    )
    flur_dua_hf_podm,ws_dua_hf_podm =(
        np.load(f'NumpyArrays\\flur_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_W_{int(w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_W_{int(w_p_odmr)}.npy')
    )

### Plot CW-ODMR

In [ ]:
norm_flur=max_reg(flur_codm)
norm_flur_hf=max_reg(flur_doh_hf_codm)
plt.figure(figsize=(14, 8))
if len(flur_codm)>30:
    y,x1,x2=min(norm_flur),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
    plt.hlines(y,x1,x2,"r","--")
    plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                            ha='center', va='bottom',color='red',fontsize=20)
plt.title("CWODMR",fontsize=20)
plt.xlabel("Frequency (MHz)",fontsize=20)
plt.ylabel("Normalized PL (a.u.)",fontsize=20)
plt.plot(ws_codm,norm_flur,label="No-HF",color="orange")
plt.plot(ws_doh_hf_codm,norm_flur_hf,label="HF",ls="--")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.minorticks_on()

In [ ]:
if save_odmr and run_odmr:
    np.save(f'NumpyArrays\\flur_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',flur_codm)
    np.save(f'NumpyArrays\\ns_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',ns_codm)
    np.save(f'NumpyArrays\\ts_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',ts_codm)
    np.save(f'NumpyArrays\\ns_hf_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',ns_hf_codm)
    np.save(f'NumpyArrays\\ts_hf_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',ts_hf_codm)
    np.save(f'NumpyArrays\\flur_hf_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',flur_hf_codm)
    np.save(f'NumpyArrays\\ws_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',ws_codm)
    np.save(f'NumpyArrays\\ws_hf_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',ws_hf_codm)

### Run Contrast

In [ ]:
if run_contrast:
    if b_odmr<600:
        n,m,s=2/5,25,10
        eps=n*b_odmr
    else:
        n,m,s=1/5,50,20
        eps=n*b_odmr
    wslhfl=np.linspace(D_gs-mu_e*b_odmr-mu_e*b_odmr*n,D_gs-mu_e*b_odmr-1.5*a_gs[0],s)
    wslhf=np.linspace(D_gs-mu_e*b_odmr-1.5*a_gs[0],D_gs-mu_e*b_odmr+1.5*a_gs[0],m)
    wslhfh=np.linspace(D_gs-mu_e*b_odmr+1.5*a_gs[0],D_gs-mu_e*b_odmr+mu_e*b_odmr*n,s)
    wsm=np.linspace(D_gs-mu_e*b_odmr+mu_e*b_odmr*n+eps,D_gs+mu_e*b_odmr-mu_e*b_odmr*n-eps,2*s)
    wshhfl=np.linspace(D_gs+mu_e*b_odmr-mu_e*b_odmr*n,D_gs+mu_e*b_odmr-1.5*a_gs[0],s)
    wshhf=np.linspace(D_gs+mu_e*b_odmr-1.5*a_gs[0],D_gs+mu_e*b_odmr+1.5*a_gs[0],m)
    wshhfh=np.linspace(D_gs+mu_e*b_odmr+1.5*a_gs[0],D_gs+mu_e*b_odmr+mu_e*b_odmr*n,s)
    ws=np.unique(np.hstack((wslhfl,wslhf,wslhfh,wsm,wshhfl,wshhf,wshhfh)))
    _,cont_codm,cont_ts_codm,cont_ns_codm,_=cont_cwodmr(10.0,ws,dynamics_zfs,
                                                        1/3*(n1+n2+n3),b=B0(b_odmr,0.0,0.0))
    _,cont_hf_codm,cont_ts_hf_codm,cont_ns_hf_codm,_=cont_cwodmr(10.0,ws,dynamics_zfs_hf,
                                                                 (1/3*(n1+n2+n3))&(1/2*IdN15),b=B0(b_odmr,0.0,0.0))
else:
    cont_codm,ws_codm = np.load(f'NumpyArrays\\cont_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy'),np.load(f'NumpyArrays\\ws_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy')
    cont_hf_codm,ws_hf_codm = np.load (f'NumpyArrays\\cont_hf_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy'),np.load(f'NumpyArrays\\ws_hf_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy')
    cont_podm,ws_podm = np.load (f'NumpyArrays\\cont_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy'),np.load(f'NumpyArrays\\ws_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy')
    cont_hf_podm,ws_hf_podm = np.load (f'NumpyArrays\\cont_hf_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy'),np.load(f'NumpyArrays\\ws_hf_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy')

### Plot Contrast

In [ ]:
cont_flur_codm=(flur_codm-cont_codm)/cont_codm
cont_flur_hf_codm=(flur_hf_codm-cont_hf_codm)/cont_hf_codm
plt.figure(figsize=(14, 8))
if len(flur_codm)>30:
    y,x1,x2=min(cont_flur_codm),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
    plt.hlines(y,x1,x2,"r","--")
    plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                            ha='center', va='bottom',color='r',fontsize=20)
plt.plot(ws_codm,cont_flur_codm,label="No-HF",color="orange")
plt.title("CWODMR",fontsize=20)
plt.xlabel("Frequency (MHz)",fontsize=20)
plt.ylabel("Contrast",fontsize=20)
plt.plot(ws_hf_codm,cont_flur_hf_codm,label="HF",ls="--")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.minorticks_on()

In [ ]:
if save_contrast and run_contrast:
    np.save(f'NumpyArrays\\cont_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',cont_codm)
    np.save(f'NumpyArrays\\cont_ns_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',cont_ns_codm)
    np.save(f'NumpyArrays\\cont_hf_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',cont_hf_codm)
    np.save(f'NumpyArrays\\cont_ns_hf_cwodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',cont_ns_hf_codm)

## Pulsed ODMR (PuODMR)

In [ ]:
def puodmr(tp, tw, tr, ws, model, init_state, b=B, om_r=Om_r, w_p=W_p):
    """
    Perform a series of simulations for the puodmr experiment.
    
    Args:
        tp (float): Laser pulse duration.
        tw (float): Free evolution time.
        tr (float): Laser pulse duration.
        ws (array-like): Array of angular frequencies.
        model (function): Function that performs the simulation.
        init_state (object): Initial state of the system.
        b (float, optional): Magnetic field strength. Defaults to B.
        om_r (float, optional): Rabi frequency. Defaults to Om_r.
        w_p (float, optional): Probe frequency. Defaults to W_p.
    
    Returns:
        tuple: A tuple containing the following elements:
            - ws (array-like): Array of angular frequencies.
            - flur (array-like): Array of PL values.
            - times (array-like): Array of time values for the plots.
            - n_exps (array-like): Array of expectation values for each operator.
            - res (array-like): Array of simulation results.
    """
    flur=np.array([])
    for w in tqdm(ws):
        results=np.array([])
        time=np.array([])
        ti=0.0
        #laser tp
        tf,result=model(tp, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=W_p, 
                        ti=ti, 
                        mode="Laser", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #free tw
        ti=tf
        tf,result=model(tw, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="Free", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #pi pulse
        ti=tf
        tpi=np.pi/om_r
        tf,result=model(tpi, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p, 
                        ti=ti, 
                        mode="MW", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #laser tr
        ti=tf
        tf,result=model(tr, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p, 
                        ti=ti, 
                        mode="Laser", 
                        progress_bar="OFF")
        results=np.append(results,result)
        for j in range(len(results[0].expect)):
            nn_exp = np.array([])
            for i in range(len(results)):
                nn_exp = np.append(nn_exp, results[i].expect[j])
            if j == 0:
                n_exp = np.array([nn_exp])
            else:
                n_exp = np.append(n_exp, [nn_exp], axis=0)
        fl=n_exp[3]+n_exp[4]+n_exp[5]
        flur=np.append(flur,np.sum(fl))
        # Gather the times for the plots
        for i in range(len(results)):
            time = np.append(time, results[i].times)
        if w == ws[0]:
            times=np.array([time])
            res=np.array([results])
            n_exps=np.array([n_exp])
        else:    
            times=np.append(times, [time], axis=0)
            res=np.append(res, [results], axis=0)
            n_exps=np.append(n_exps, [n_exp], axis=0)
    return ws,flur,times,n_exps,res

### Contrast for PuODMR

In [ ]:
def cont_puodmr(tp, tw, tr, ws, model, init_state, b=B, om_r=Om_r, w_p=W_p):
    """
    Calculate the contrast of a PuODMR (Pulsed Optically Detected Magnetic Resonance) signal.
    
    Parameters:
    - tp (float): Laser pulse duration for the first laser pulse.
    - tw (float): Free evolution time between the first and second laser pulses.
    - tr (float): Laser pulse duration for the second laser pulse.
    - ws (array-like): Array of angular frequencies.
    - model (function): Function that models the system dynamics_zfs.
    - init_state (object): Initial state of the system.
    - b (float, optional): Magnetic field strength. Defaults to B.
    - om_r (float, optional): Resonance frequency. Defaults to Om_r.
    - w_p (float, optional): Rabi frequency. Defaults to W_p.
    
    Returns:
    - ws (array-like): Array of angular frequencies.
    - flur (array-like): Array of contrast values.
    - times (array-like): Array of time values for the plots.
    - n_exps (array-like): Array of expectation values for each angular frequency.
    - res (array-like): Array of simulation results for each angular frequency.
    """
    flur=np.array([])
    for w in tqdm(ws):
        results=np.array([])
        time=np.array([])
        ti=0.0
        #laser tp
        tf,result=model(tp, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=W_p,
                        ti=ti, 
                        mode="Laser", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #free tw
        ti=tf
        tf,result=model(tw, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="Free", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #pi pulse
        ti=tf
        tpi=np.pi/om_r
        tf,result=model(tpi, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="Free", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #laser tr
        ti=tf
        tf,result=model(tr, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="Laser", 
                        progress_bar="OFF")
        results=np.append(results,result)
        for j in range(len(results[0].expect)):
            nn_exp = np.array([])
            for i in range(len(results)):
                nn_exp = np.append(nn_exp, results[i].expect[j])
            if j == 0:
                n_exp = np.array([nn_exp])
            else:
                n_exp = np.append(n_exp, [nn_exp], axis=0)
        fl=n_exp[3]+n_exp[4]+n_exp[5]
        flur=np.append(flur,np.sum(fl))
        # Gather the times for the plots
        for i in range(len(results)):
            time = np.append(time, results[i].times)
        if w == ws[0]:
            times=np.array([time])
            res=np.array([results])
            n_exps=np.array([n_exp])
        else:    
            times=np.append(times, [time], axis=0)
            res=np.append(res, [results], axis=0)
            n_exps=np.append(n_exps, [n_exp], axis=0)
    return ws,flur,times,n_exps,res

### Run PuODMR

In [ ]:
if run_odmr:
    if b_odmr<600:
        n,m,s=2/5,25,10
        eps=n*b_odmr
    else:
        n,m,s=1/5,50,20
        eps=n*b_odmr
    tI,tR=10,7
    wslhfl=np.linspace(D_gs-mu_e*b_odmr-mu_e*b_odmr*n,D_gs-mu_e*b_odmr-1.5*a_gs[0],s+s//4+1)
    wslhf=np.linspace(D_gs-mu_e*b_odmr-1.5*a_gs[0],D_gs-mu_e*b_odmr+1.5*a_gs[0],m)
    wslhfh=np.linspace(D_gs-mu_e*b_odmr+1.5*a_gs[0],D_gs-mu_e*b_odmr+mu_e*b_odmr*n,s+s//4)
    wsm=np.linspace(D_gs-mu_e*b_odmr+mu_e*b_odmr*n+eps,D_gs+mu_e*b_odmr-mu_e*b_odmr*n-eps,s)
    wshhfl=np.linspace(D_gs+mu_e*b_odmr-mu_e*b_odmr*n,D_gs+mu_e*b_odmr-1.5*a_gs[0],s+s//4)
    wshhf=np.linspace(D_gs+mu_e*b_odmr-1.5*a_gs[0],D_gs+mu_e*b_odmr+1.5*a_gs[0],m)
    wshhfh=np.linspace(D_gs+mu_e*b_odmr+1.5*a_gs[0],D_gs+mu_e*b_odmr+mu_e*b_odmr*n,s+s//4+1)
    ws=np.unique(np.hstack((wslhfl,wslhf,wslhfh,wsm,wshhfl,wshhf,wshhfh)))
    ws_podm,flur_podm,ts_podm,ns_podm,_=puodmr(tI,1.0,tR,ws,dynamics_zfs,
                                               1/3*(n1+n2+n3),b=B0(b_odmr,0.0,0.0))
    ws_hf_podm,flur_hf_podm,ts_hf_podm,ns_hf_podm,_=puodmr(tI,1.0,tR,ws,dynamics_zfs_hf,
                                                           (1/3*(n1+n2+n3))&(1/2*IdN15),b=B0(b_odmr,0.0,0.0))

### Plot PuODMR

In [ ]:
norm_flur=max_reg(flur_podm)
norm_flur_hf=max_reg(flur_doh_hf_podm)
plt.figure(figsize=(14, 8))
if len(flur_podm)>30:
    y,x1,x2=min(norm_flur),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
    plt.hlines(y,x1,x2,"r","--")
    plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                            ha='center', va='bottom',color='r',fontsize=20)
plt.plot(ws_podm,norm_flur,label="No-HF",color="orange")
plt.title("PUODMR",fontsize=20)
plt.xlabel("Frequency (MHz)",fontsize=20)
plt.ylabel("Normalized PL (a.u.)",fontsize=20)
plt.plot(ws_doh_hf_podm,norm_flur_hf,label="HF",ls="--")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.minorticks_on()

In [ ]:
if save_odmr and run_odmr:
    np.save(f'NumpyArrays\\flur_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',flur_podm)
    np.save(f'NumpyArrays\\ns_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',ns_podm)
    np.save(f'NumpyArrays\\ts_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',ts_podm)
    np.save(f'NumpyArrays\\ns_hf_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',ns_hf_podm)
    np.save(f'NumpyArrays\\ts_hf_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',ts_hf_podm)
    np.save(f'NumpyArrays\\flur_hf_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',flur_hf_podm)
    np.save(f'NumpyArrays\\ws_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',ws_podm)
    np.save(f'NumpyArrays\\ws_hf_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',ws_hf_podm)

### Run Contrast

In [ ]:
if run_contrast:
    if b_odmr<600:
        n,m,s=2/5,25,10
        eps=n*b_odmr
    else:
        n,m,s=1/5,50,20
        eps=n*b_odmr
    wslhfl=np.linspace(D_gs-mu_e*b_odmr-mu_e*b_odmr*n,D_gs-mu_e*b_odmr-1.5*a_gs[0],s+s//4+1)
    wslhf=np.linspace(D_gs-mu_e*b_odmr-1.5*a_gs[0],D_gs-mu_e*b_odmr+1.5*a_gs[0],m)
    wslhfh=np.linspace(D_gs-mu_e*b_odmr+1.5*a_gs[0],D_gs-mu_e*b_odmr+mu_e*b_odmr*n,s+s//4)
    wsm=np.linspace(D_gs-mu_e*b_odmr+mu_e*b_odmr*n+eps,D_gs+mu_e*b_odmr-mu_e*b_odmr*n-eps,s)
    wshhfl=np.linspace(D_gs+mu_e*b_odmr-mu_e*b_odmr*n,D_gs+mu_e*b_odmr-1.5*a_gs[0],s+s//4)
    wshhf=np.linspace(D_gs+mu_e*b_odmr-1.5*a_gs[0],D_gs+mu_e*b_odmr+1.5*a_gs[0],m)
    wshhfh=np.linspace(D_gs+mu_e*b_odmr+1.5*a_gs[0],D_gs+mu_e*b_odmr+mu_e*b_odmr*n,s+s//4+1)
    ws=np.unique(np.hstack((wslhfl,wslhf,wslhfh,wsm,wshhfl,wshhf,wshhfh)))
    _,cont_podm,cont_ts_podm,cont_ns_podm,_=cont_puodmr(tI,1.0,tR,ws,dynamics_zfs,
                                                        1/3*(n1+n2+n3),b=B0(b_odmr,0.0,0.0))
    _,cont_hf_podm,cont_ts_hf_podm,cont_ns_hf_podm,_=cont_puodmr(tI,1.0,tR,ws,dynamics_zfs_hf,
                                                                 (1/3*(n1+n2+n3))&(1/2*IdN15),b=B0(b_odmr,0.0,0.0))

### Plot Contrast

In [ ]:
cont_flur_podm=(flur_podm-cont_podm)/cont_podm
cont_flur_hf_podm=(flur_hf_podm-cont_hf_podm)/cont_hf_podm
plt.figure(figsize=(14, 8))
if len(flur_podm)>30:
    y,x1,x2=min(cont_flur_podm),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
    plt.hlines(y,x1,x2,"r","--")
    plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                            ha='center', va='bottom',color='r',fontsize=20)
plt.plot(ws_podm,cont_flur_podm,label="No-HF",color="orange")
plt.title("PUODMR",fontsize=20)
plt.xlabel("Frequency (MHz)",fontsize=20)
plt.ylabel("Contrast",fontsize=20)
plt.plot(ws_hf_podm,cont_flur_hf_podm,label="HF",ls="--")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.minorticks_on()

In [ ]:
if save_contrast and run_contrast:
    np.save(f'NumpyArrays\\cont_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',cont_podm)
    np.save(f'NumpyArrays\\cont_ns_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',cont_ns_podm)
    np.save(f'NumpyArrays\\cont_hf_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',cont_hf_podm)
    np.save(f'NumpyArrays\\cont_ns_hf_puodmr_B_{int(b_odmr)}_om_{int(Om_r)}_a_{int(a_gs[0])}.npy',cont_ns_hf_podm)

## Comparisson CW-ODMR and PuODMR

In [ ]:
plt.figure(figsize=(14, 8))
plt.xlabel("Frequency (MHz)")
plt.ylabel("Normalized PL (a.u.)")
plt.plot(ws_codm,max_reg(flur_codm),label="no-HF,CWODMR")
plt.plot(ws_doh_hf_codm,max_reg(flur_doh_hf_codm),label="HF,CWODMR",ls="--",lw=2)
plt.plot(ws_podm,max_reg(flur_podm),label="no-HF,PUODMR")
plt.plot(ws_doh_hf_podm,max_reg(flur_doh_hf_podm),label="HF,PUODMR",ls="--",lw=2)
all_fl=np.append(max_reg(flur_hf_podm),max_reg(flur_podm))
all_fl=np.append(all_fl,max_reg(flur_hf_codm))
all_fl=np.append(all_fl,max_reg(flur_codm))
y,x1,x2=min(all_fl),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
plt.hlines(y,x1,x2,"r","dotted")
plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                        ha='center', va='bottom',color='r',fontsize=20)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.minorticks_on()

In [ ]:
if b_odmr<600:    
    lims=2*len(flur_codm)//17,5*len(flur_codm)//17
else:
    lims=2*len(flur_codm)//34,10*len(flur_codm)//34
s=10
plt.xlabel("Frequency (MHz)")
plt.ylabel("Contrast")
norm_cont_codm=(flur_codm[lims[0]:lims[1]]-cont_codm[lims[0]:lims[1]])/cont_codm[lims[0]:lims[1]]
norm_cont_codm_hf=(flur_hf_codm[lims[0]:lims[1]]-cont_hf_codm[lims[0]:lims[1]])/cont_hf_codm[lims[0]:lims[1]]
norm_cont_podm=(flur_podm[lims[0]:lims[1]+s//2]-cont_podm[lims[0]:lims[1]+s//2])/cont_podm[lims[0]:lims[1]+s//2]
norm_cont_podm_hf=(flur_hf_podm[lims[0]:lims[1]+s//2]-cont_hf_podm[lims[0]:lims[1]+s//2])/cont_hf_podm[lims[0]:lims[1]+s//2]
y,x1,x2=min(np.append(norm_cont_podm_hf,norm_cont_codm_hf)),D_gs-mu_e*b_odmr-a_gs[0]/2,D_gs-mu_e*b_odmr+a_gs[0]/2
plt.hlines(y,x1,x2,"r","dotted")
gs="{gs}"
para="{||}"
plt.annotate(f"$A^{gs}_{para}={round(a_gs[0],3)}$ MHz",xy=(D_gs-mu_e*b_odmr,y),
             xytext=(0,-22),textcoords='offset points',ha='center', va='bottom',color='r',fontsize=12)
plt.plot(ws_codm[lims[0]:lims[1]],norm_cont_codm,label="no-HF,CWODMR")
plt.plot(ws_hf_codm[lims[0]:lims[1]],norm_cont_codm_hf,label="HF,CWODMR",ls="--",lw=2)
plt.plot(ws_podm[lims[0]:lims[1]+s//2],norm_cont_podm,label="no-HF,PUODMR")
plt.plot(ws_hf_podm[lims[0]:lims[1]+s//2],norm_cont_podm_hf,label="HF,PUODMR",ls="--",lw=2)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.minorticks_on()

In [ ]:
if b_odmr<600:    
    lims=11*len(flur_codm)//17,15*len(flur_codm)//17
else:
    lims=22*len(flur_codm)//34,30*len(flur_codm)//34
plt.xlabel("Frequency (MHz)")
plt.ylabel("Contrast")
norm_cont_codm=(flur_codm[lims[0]+s:lims[1]]-cont_codm[lims[0]+s:lims[1]])/cont_codm[lims[0]+s:lims[1]]
norm_cont_codm_hf=(flur_hf_codm[lims[0]+s:lims[1]]-cont_hf_codm[lims[0]+s:lims[1]])/cont_hf_codm[lims[0]+s:lims[1]]
norm_cont_podm=(flur_podm[lims[0]+s//2:lims[1]]-cont_podm[lims[0]+s//2:lims[1]])/cont_podm[lims[0]+s//2:lims[1]]
norm_cont_podm_hf=(flur_hf_podm[lims[0]+s//2:lims[1]]-cont_hf_podm[lims[0]+s//2:lims[1]])/cont_hf_podm[lims[0]+s//2:lims[1]]
y,x1,x2=min(np.append(norm_cont_podm_hf,norm_cont_codm_hf)),D_gs+mu_e*b_odmr-a_gs[0]/2,D_gs+mu_e*b_odmr+a_gs[0]/2
plt.hlines(y,x1,x2,"r","dotted")
gs="{gs}"
para="{||}"
plt.annotate(f"$A^{gs}_{para}={round(a_gs[0],3)}$ MHz",xy=(D_gs+mu_e*b_odmr,y),
             xytext=(0,-22),textcoords='offset points',ha='center', va='bottom',color='r',fontsize=12)
plt.plot(ws_codm[lims[0]+s:lims[1]],norm_cont_codm,label="no-HF,CWODMR")
plt.plot(ws_hf_codm[lims[0]+s:lims[1]],norm_cont_codm_hf,label="HF,CWODMR",ls="--",lw=2)
plt.plot(ws_podm[lims[0]+s//2:lims[1]],norm_cont_podm,label="no-HF,PUODMR")
plt.plot(ws_hf_podm[lims[0]+s//2:lims[1]],norm_cont_podm_hf,label="HF,PUODMR",ls="--",lw=2)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.minorticks_on()

In [ ]:
if b_odmr<600:    
    lims=6*len(flur_codm)//14,8*len(flur_codm)//14
else:
    lims=11*len(flur_codm)//28,16*len(flur_codm)//28
plt.xlabel("Frequency (MHz)")
plt.ylabel("Contrast")
norm_cont_codm=(flur_codm[lims[0]:lims[1]+s//4]-cont_codm[lims[0]:lims[1]+s//4])/cont_codm[lims[0]:lims[1]+s//4]
norm_cont_codm_hf=(flur_hf_codm[lims[0]:lims[1]+s//4]-cont_hf_codm[lims[0]:lims[1]+s//4])/cont_hf_codm[lims[0]:lims[1]+s//4]
norm_cont_podm=(flur_podm[lims[0]+s//3:lims[1]-s//4]-cont_podm[lims[0]+s//3:lims[1]-s//4])/cont_podm[lims[0]+s//3:lims[1]-s//4]
norm_cont_podm_hf=(flur_hf_podm[lims[0]+s//3:lims[1]-s//4]-cont_hf_podm[lims[0]+s//3:lims[1]-s//4])/cont_hf_podm[lims[0]+s//3:lims[1]-s//4]
plt.plot(ws_codm[lims[0]:lims[1]+s//4],norm_cont_codm,label="no-HF,CWODMR")
plt.plot(ws_hf_codm[lims[0]:lims[1]+s//4],norm_cont_codm_hf,label="HF,CWODMR",ls="--",lw=2)
plt.plot(ws_podm[lims[0]+s//3:lims[1]-s//4],norm_cont_podm,label="no-HF,PUODMR")
plt.plot(ws_hf_podm[lims[0]+s//3:lims[1]-s//4],norm_cont_podm_hf,label="HF,PUODMR",ls="--",lw=2)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.minorticks_on()

### Some odd things

#### Beginning dent
This is just some strange error from qutip

In [ ]:
ws_l_podm,flur_l_podm=np.load('NumpyArrays\\dents\\ws_beg_dent_B_500_om_15.npy'),np.load('NumpyArrays\\dents\\fl_beg_dent_B_500_om_15.npy')
ws_l_hf_podm,flur_l_hf_podm=np.load('NumpyArrays\\dents\\ws_hf_beg_dent_B_500_om_15.npy'),np.load('NumpyArrays\\dents\\fl_hf_beg_dent_B_500_om_15.npy')
plt.xlabel("Frequency (MHz)")
plt.ylabel("Contrast")
plt.title("PUODMR",fontsize=20)
plt.plot(ws_l_podm,flur_l_podm,label="no-HF")
plt.plot(ws_l_hf_podm,flur_l_hf_podm,label="HF",ls='--')
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.minorticks_on()

#### Middle dent
This happens only when B is near 500G

In [ ]:
ws_m_codm,flur_m_codm=np.load('NumpyArrays\\dents\\ws_m_dent_B_500_om_15.npy'),np.load  ('NumpyArrays\\dents\\fl_m_dent_B_500_om_15.npy')
ws_hf_m_codm,flur_m_hf_codm=np.load('NumpyArrays\\dents\\ws_hf_m_dent_B_500_om_15.npy'),np.load  ('NumpyArrays\\dents\\fl_hf_m_dent_B_500_om_15.npy')
plt.xlabel("Frequency (MHz)")
plt.title("Middle Dent CWODMR",fontsize=20)
plt.plot(ws_m_codm,flur_m_codm/np.max(flur_m_codm),label="no-HF")
plt.plot(ws_hf_m_codm,flur_m_hf_codm/np.max(flur_m_hf_codm),label="HF",ls='--')
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.minorticks_on()

## Ramsey

In [ ]:
def ramsey(tp, tw, ts, tr, model, init_state,
           b=B, om_r=Om_r, w_p=W_p):
    """
    Perform Ramsey experiment simulation.
    
    Parameters:
    - tp (float): Laser pulse duration for the first pi/2 pulse.
    - tw (float): Free evolution time between the first and second pi/2 pulses.
    - ts (float): Free evolution time between the second pi/2 pulse and the Ramsey pulse.
    - tr (float): Laser pulse duration for the Ramsey pulse.
    - model (function): Function that defines the simulation model.
    - init_state (object): Initial state of the system.
    - b (float): Parameter b.
    - om_r (float): Rabi frequency.
    - w_p (float): Parameter w_p.
    
    Returns:
    - ws (array): Array of frequencies.
    - flur (array): Array of PL values.
    - times (array): Array of times for the plots.
    - n_exps (array): Array of expectation values.
    - res (array): Array of simulation results.
    """
    flur=np.array([])
    for w in tqdm(ws):
        results=np.array([])
        time=np.array([])
        ti=0.0
        #laser tp
        tf,result=model(tp, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p, 
                        ti=ti, 
                        mode="Laser", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #free tw
        ti=tf
        tf,result=model(tw, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="Free", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #pi/2 pulse
        ti=tf
        tpi2=0.5*np.pi/om_r
        tf,result=model(tpi2, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="MW", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #free ts
        ti=tf
        tf,result=model(ts, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="Free", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #pi/2 pulse
        ti=tf
        tpi2=0.5*np.pi/om_r
        tf,result=model(tpi2, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="MW", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #laser tr
        ti=tf
        tf,result=model(tr, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="Laser", 
                        progress_bar="OFF")
        results=np.append(results,result)
        for j in range(len(results[0].expect)):
            nn_exp = np.array([])
            for i in range(len(results)):
                nn_exp = np.append(nn_exp, results[i].expect[j])
            if j == 0:
                n_exp = np.array([nn_exp])
            else:
                n_exp = np.append(n_exp, [nn_exp], axis=0)
        fl=n_exp[3]+n_exp[4]+n_exp[5]
        flur=np.append(flur,np.sum(fl))
        # Gather the times for the plots
        for i in range(len(results)):
            time = np.append(time, results[i].times)
        if w == ws[0]:
            times=np.array([time])
            res=np.array([results])
            n_exps=np.array([n_exp])
        else:    
            times=np.append(times, [time], axis=0)
            res=np.append(res, [results], axis=0)
            n_exps=np.append(n_exps, [n_exp], axis=0)
    return ws,flur,times,n_exps,res
#ts=t2 no-hf and ts=2pi/a_para hf

## Spin-Echo

In [ ]:
def spin_echo(tp, tw, ts1, ts2, tr, model, init_state,
              b=B, om_r=Om_r, w_p=W_p):
    """
    Perform a spin echo experiment.
    
    Parameters:
    - tp (float): Duration of the laser pulse.
    - tw (float): Duration of the free evolution period.
    - ts1 (float): Duration of the first free evolution period.
    - ts2 (float): Duration of the second free evolution period.
    - tr (float): Duration of the laser pulse.
    - model (function): Function that models the system dynamics_zfs.
    - init_state (object): Initial state of the system.
    - b (float): Magnetic field strength (default: B).
    - om_r (float): Resonance frequency (default: Om_r).
    - w_p (float): Rabi frequency (default: W_p).
    
    Returns:
    - ws (array): Array of frequencies.
    - flur (array): Array of PL values.
    - times (array): Array of time values.
    - n_exps (array): Array of expectation values.
    - res (array): Array of simulation results.
    """
    flur=np.array([])
    for w in tqdm(ws):
        results=np.array([])
        time=np.array([])
        ti=0.0
        #laser tp
        tf,result=model(tp, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="Laser", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #free tw
        ti=tf
        tf,result=model(tw, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="Free", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #pi/2 pulse
        ti=tf
        tpi2=0.5*np.pi/om_r
        tf,result=model(tpi2, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p, 
                        ti=ti, 
                        mode="MW", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #free ts1
        ti=tf
        tf,result=model(ts1, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="Free", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #pi pulse
        ti=tf
        tpi=np.pi/om_r
        tf,result=model(tpi, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="MW", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #free ts2
        ti=tf
        tf,result=model(ts2, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p, 
                        ti=ti, 
                        mode="Free", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #pi/2 pulse
        ti=tf
        tpi2=0.5*np.pi/om_r
        tf,result=model(tpi2, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="MW", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #laser tr
        ti=tf
        tf,result=model(tr, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p, 
                        ti=ti, 
                        mode="Laser", 
                        progress_bar="OFF")
        results=np.append(results,result)
        for j in range(len(results[0].expect)):
            nn_exp = np.array([])
            for i in range(len(results)):
                nn_exp = np.append(nn_exp, results[i].expect[j])
            if j == 0:
                n_exp = np.array([nn_exp])
            else:
                n_exp = np.append(n_exp, [nn_exp], axis=0)
        fl=n_exp[3]+n_exp[4]+n_exp[5]
        flur=np.append(flur,np.sum(fl))
        # Gather the times for the plots
        for i in range(len(results)):
            time = np.append(time, results[i].times)
        if w == ws[0]:
            times=np.array([time])
            res=np.array([results])
            n_exps=np.array([n_exp])
        else:    
            times=np.append(times, [time], axis=0)
            res=np.append(res, [results], axis=0)
            n_exps=np.append(n_exps, [n_exp], axis=0)
    return ws,flur,times,n_exps,res

# NV Center Orientations and Lattice Simulations

## Model definition

In [ ]:
pl=excited[0]*excited[0].dag()+excited[1]*excited[1].dag()+excited[2]*excited[2].dag() #type: ignore

In [ ]:
def photolumin(dynamics,laser_time, free_time,i_state=(1 / 3 * (n1 + n2 + n3))&(1 / 2 * IdN15),e_ops=[pl],b=B, om_r=Om_r, om=omega, w_p_i=W_p,w_p_r=3*W_p,progress_bar="ON"):
    """
    Simulates the evolution of a quantum system under a given protocol with continuos laser.

    Args:
        laser_time (float): The duration of the laser pulse in each iteration.
        mw_time (float): The duration of the microwave pulse in each iteration.
        free_time (float): The duration of the free evolution in each iteration.
        protocol_repeats (int, optional): The number of times to repeat the protocol. Defaults to 1.

    Returns:
        tuple: A tuple containing the following arrays:
            - n_exp (numpy.ndarray): An array of expectation values for different operators.
            - times (numpy.ndarray): An array of time values.
            - tis (numpy.ndarray): An array of initial times for each part of the protocol.
            - tfs (numpy.ndarray): An array of final times for each part of the protocol.
            - results (numpy.ndarray): An array of the Result class object for the time evolution.
    """
    print("Starting calculations, this may take a while...")
    c=0
    for dt_mw in tqdm([0.0,0.5*np.pi/om_r,np.pi/om_r]):
        # Initial state where the ground state is evenly populated
        init_state = i_state

        # Array to collect the result Class
        results_t = np.array([])

        # Initial time of the evolution
        ti = 0.0

        # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
        tis_t = []
        tfs_t = []

        # Set the number of times to repeat the protocol
        n = 1
        # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
        dt_laser = laser_time
        dt_free = free_time
        # This loop is defined to reproduce the protocol of Magalleti el al
        for i in tqdm(range(n),position=1,leave=False):
            tis_t.append(ti)
            # Here Laser is ON
            # Run the dynamics_doh based on the mode choosen
            tf, result = dynamics(
                dt_laser,
                init_state,
                b=b,
                om_r=om_r,
                om=om,
                w_p=w_p_i,
                exp_ops=e_ops,
                ti=ti,
                mode="Laser",
                progress_bar=progress_bar,
                i=i,
            ) # type:ignore
            # Store the Result class for when the Laser is on
            results_t = np.append(results_t, result) # type: ignore 
            # Save the initial state for the next part of the protocol
            init_state = result.states[-1]
            # Make the initial time the end time of the previous part of the procotol
            ti = tf
            tfs_t.append(tf)
            # Run the dynamics_doh based on the mode choosen
            tf, result = dynamics(
                dt_free,
                init_state,
                b=b,
                om_r=om_r,
                om=om,
                w_p=0.0,
                exp_ops=e_ops,
                ti=ti,
                mode="Free",
                progress_bar=progress_bar,
                i=i,
            ) # type:ignore
            # Store the Result class for MW on evolution
            results_t = np.append(results_t, result) # type: ignore 
            # Save the times for coloring areas of the plot
            tis_t.append(ti)
            tfs_t.append(tf)
            # Make initial time to be the end time of the last part of the protocol
            ti = tf
            # Make the last state  of this evolution step be the initial states of the next iteration
            init_state = result.states[-1]
            # Run the dynamics_doh based on the mode choosen
            tf, result = dynamics(dt_mw, 
                                  init_state,
                                  b=b,
                                  om_r=om_r,
                                  om=om,
                                  w_p=0.0,
                                  exp_ops=e_ops, 
                                  ti=ti, 
                                  mode="MW", 
                                  progress_bar=progress_bar, 
                                  i=i) # type:ignore
            # Store the Result class for MW on evolution
            results_t = np.append(results_t, result) # type: ignore
            # Save the times for coloring areas of the plot
            tis_t.append(ti)
            tfs_t.append(tf)
            # Make initial time to be the end time of the last part of the protocol
            ti = tf
            # Make the last state  of this evolution step be the initial states of the next iteration
            init_state = result.states[-1]
            tis_t.append(ti)
            # Here Laser is ON
            # Run the dynamics_doh based on the mode choosen
            tf, result = dynamics(
                dt_laser,
                init_state,
                b=b,
                om_r=om_r,
                om=om,
                w_p=w_p_r,
                exp_ops=e_ops,
                ti=ti,
                mode="Laser",
                progress_bar=progress_bar,
                i=i+1,
            ) # type:ignore
            # Store the Result class for when the Laser is on
            results_t = np.append(results_t, result) # type: ignore 
            # Save the initial state for the next part of the protocol
            init_state = result.states[-1]
            # Make the initial time the end time of the previous part of the procotol
            ti = tf
            tfs_t.append(tf)
        # Gather the expectation values from the results array
        for j in range(len(results_t[0].expect)):
            nn_exp = np.array([])
            for i in range(len(results_t)):
                nn_exp = np.append(nn_exp, results_t[i].expect[j])
            if j == 0:
                n_exp_t = np.array([nn_exp])
            else:
                n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
        # Gather the times for the plots
        times_t = np.array([])
        for i in range(len(results_t)):
            times_t = np.append(times_t, results_t[i].times)
        if c==0:
            plss=np.array([n_exp_t])
            c+=1
        else:
            plss=np.append(plss,[n_exp_t],axis=0)
    print(f"Calculations are finished!! See the plots below")
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "gold",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "mediumvioletred",
        "forestgreen",
    ]
    # Defines figure size
    plt.figure(figsize=(14, 8))
    a=0
    state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
    for pl in plss:
        plt.plot(times_t[-5200:-2500]-11,pl[0][-5200:-2500], color=colors[a], label=f"PL prepared state=${state[a]}$")
        a+=1
    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    n = 0
    aa=plss[0][-5200:]
    eps_y = abs(np.min(aa) - np.max(aa))*5e-2
    eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
    plt.ylabel(f"PL(a.u.)",fontsize=20)
    plt.xlabel(r"Time($\mu$s)",fontsize=20)
    plt.title(f"B={b[2]:.2f} G, $\\Omega_r$={om_r:.2f} MHz, \n $W_p^{{I}}$={w_p_i:.2f},$W_p^{{R}}$={w_p_r:.2f} ",fontsize=20)

    # set the time limits you want to show in the plot
    t_lim = (times_t[-5200]-11-eps_x, times_t[-2500] + eps_x-11)
    plt.xlim(t_lim)
    plt.ylim((0, np.max(aa) + eps_y))
    plt.minorticks_on()
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=14)
    plt.show()

In [ ]:
# Dimention of the Hilbert space of the NV center
dim = 7

t1_n=1e5
t2_n=1e3
# Constants (MHz)
gamma_n=1/t1_n,1/t2_n
mu_n = 431.7*1e-6 # (MHz/G)

A_par = 3.4,-58.1
A_perp = 7.8,-77.0
A_ani = 0.0, 0.0
A_perp_prime = 0.0,0.0
phi_H = 0.0

def H_dua_hf(b, om_r,phi_h):
    """
    Calculate the Hamiltonian for a given magnetic field and resonant frequency, including hyperfine interactions.
    
    Parameters:
    b (numpy.ndarray): Magnetic field vector [bx, by, bz].
    om_r: Resonant frequency.
    
    Returns:
    list: List of Hamiltonian terms.
    """
    H_0 = [(D_gs*sz_gs**2 + E*(sy_gs**2 - sx_gs**2) + mu_e*np.dot(b, S_gs))&IdN15,
           (D_es*sz_es**2 + mu_e*np.dot(b, S_es))&IdN15,
           A_par[0]*(sz_gs&sz_n) + A_perp[0]/4*((sp_gs&sm_n)+(sm_gs&sp_n)),
           A_perp_prime[0]/4*((sp_gs&sp_n)*np.exp(-2j*phi_h) + (sm_gs&sm_n)*np.exp(2j*phi_h)),
           A_ani[0]/2*((sp_gs&sz_n) + (sz_gs&sp_n)*np.exp(-1j*phi_h) + (sm_gs&sz_n) + (sz_gs&sm_n)*np.exp(1j*phi_h)),
           A_par[1]*(sz_es&sz_n) + A_perp[1]/4*((sp_es&sm_n)+(sm_es&sp_n)),
           A_perp_prime[1]/4*((sp_es&sp_n)*np.exp(-2j*phi_h) + (sm_es&sm_n)*np.exp(2j*phi_h)),
           A_ani[1]/2*((sp_es&sz_n) + (sz_es&sp_n)*np.exp(-1j*phi_h) + (sm_es&sz_n) + (sz_es&sm_n)*np.exp(1j*phi_h))]
    H_n = [IdNV&mu_n*np.dot(b, S_n)]
    H_int = [[(np.sqrt(2)*om_r*sx_gs)&IdN15, "cos(w*t)*cos(p)"],
             [(np.sqrt(2)*om_r*sy_gs)&IdN15, "cos(w*t)*sin(p)"],
             [(np.sqrt(2)*om_r*sx_es)&IdN15, "cos(w*t)*cos(p)"],
             [(np.sqrt(2)*om_r*sy_es)&IdN15, "cos(w*t)*sin(p)"],
             [IdNV&(2*om_r/mu_e*mu_n*sx_n), "cos(w*t)*cos(p)"],
             [IdNV&(2*om_r/mu_e*mu_n*sy_n), "cos(w*t)*sin(p)"]]
    return [*H_0, *H_n, *H_int]

def L_dua_hf(w_p,k_index=k):
    """
    Returns the Lindblad operators of the system, including optical transitions based on the given k_index.

    Parameters:
    - w_p (float): Laser pump rate.
    - k_index (int, optional): Index for the optical transition rates. Defaults to k.

    Returns:
    - c_ops (list): List of Lindblad operators.
    """
    K_s=[[66.0,0.0,57.0,1.0,0.7],
         [77.0,0.0,30.0,3.3,0.0],
         [62.7,12.97,80.0,3.45,1.08],
         [63.2,10.8,60.7,0.8,0.4],
         [67.4,9.9,96.6,4.83,1.055]]
    k41 = K_s[k_index][0]
    k52 = K_s[k_index][0]
    k63 = K_s[k_index][0]
    k57 = K_s[k_index][2]
    k67 = K_s[k_index][2]
    k47 = K_s[k_index][1]
    k71 = K_s[k_index][3]
    k72 = K_s[k_index][4]
    k73 = K_s[k_index][4]
    
    c_ops = []

    c_ops.append((np.sqrt(w_p) * (excited[1] * ground[1].dag()))&IdN15)  # n1 to n4 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[2] * ground[2].dag()))&IdN15)  # n2 to n5 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[0] * ground[0].dag()))&IdN15) # n3 to n6 #type: ignore

    c_ops.append((np.sqrt(k41) * (ground[1] * excited[1].dag()))&IdN15)  # n4 to n1 #type: ignore
    c_ops.append((np.sqrt(k71) * (ground[1] * isc.dag()))&IdN15)  # n7 to n1 #type: ignore

    c_ops.append((np.sqrt(k52) * (ground[2] * excited[2].dag()))&IdN15)  # n5 to n2 #type: ignore
    c_ops.append((np.sqrt(k72) * (ground[2] * isc.dag()))&IdN15)  # n7 to n2 #type: ignore

    c_ops.append((np.sqrt(k63) * (ground[0] * excited[0].dag()))&IdN15)  # n6 to n3 #type: ignore
    c_ops.append((np.sqrt(k73) * (ground[0] * isc.dag()))&IdN15)  # n7 to n3 #type: ignore

    c_ops.append((np.sqrt(k47) * (isc * excited[1].dag()))&IdN15)  # n4 to n7 #type: ignore
    c_ops.append((np.sqrt(k57) * (isc * excited[2].dag()))&IdN15)  # n5 to n7 #type: ignore
    c_ops.append((np.sqrt(k67) * (isc * excited[0].dag()))&IdN15)  # n6 to n7 #type: ignore
    # Add collapse operators for decoherence   
    c_ops.append((np.sqrt(gamma_gs[1]) * sz_gs)&IdN15)
    c_ops.append((np.sqrt(gamma_gs[0]/2) * (sm_gs))&IdN15)
    c_ops.append((np.sqrt(gamma_gs[0]/2) * (sp_gs))&IdN15)
    c_ops.append((np.sqrt(gamma_es[1]) * sz_es)&IdN15)
    c_ops.append((np.sqrt(gamma_es[0]/2) * (sm_es))&IdN15)
    c_ops.append((np.sqrt(gamma_es[0]/2) * (sp_es))&IdN15)
    c_ops.append(IdNV&(np.sqrt(gamma_n[1]) * sz_n))
    c_ops.append(IdNV&(np.sqrt(gamma_n[0]/2) * (sm_n)))
    c_ops.append(IdNV&(np.sqrt(gamma_n[0]/2) * (sp_n)))

    return c_ops

def dynamics_dua_hf(
    dt,
    init_state,
    b=None,
    om_r=None,
    om=None,
    p=None,
    w_p=None,
    k_index=k,
    exp_ops=None,
    ti=0.0,
    mode="Free",
    progress_bar="ON",
    i=0,
):
    """
    Simulate the dynamics of a quantum system under hyperfine interaction using the Hamiltonian and collapse operators.
    including optical transition rates index.
    Where, when using k_index=2 -> K_s[k_index]=[62.7,12.97,80.0,3.45,1.08], and the full K_s list is:
        K_s=[[66.0,0.0,57.0,1.0,0.7],
             [77.0,0.0,30.0,3.3,0.0],
             [62.7,12.97,80.0,3.45,1.08],
             [63.2,10.8,60.7,0.8,0.4],
             [67.4,9.9,96.6,4.83,1.055]]
    Parameters:
    - dt (float): Total simulation time.
    - init_state (qt.Qobj): Initial quantum state of the system.
    - b (np.array, optional): Magnetic field vector. Defaults to B0(B_amp,phi_B,theta_B).
    - om_r (float, optional): Rabi frequency for microwave interactions. Defaults to Om_r.
    - om (float, optional): Angular frequency of the system. Defaults to omega.
    - p (float, optional): Microwave phase. Defaults to 0.0.
    - w_p (float, optional): Laser frequency. Defaults to W_p.
    - k_index(int, optional): Index for the optical transition rates. Defaults to k.
    - exp_ops (list of qt.Qobj, optional): List of operators for which to compute expectation values. 
        Defaults to a list of tensor products including different Pauli operators and state projectors.
    - ti (float, optional): Initial time of the simulation. Defaults to 0.0.
    - mode (str, optional): Simulation mode. Can be "Free", "MW", "Laser", or "Laser-MW". Defaults to "Free".
    - progress_bar (str, optional): Option to display a progress bar. Can be "ON" or "OFF". Defaults to "ON".
    - i (int, optional): Counter for the progress bar. Defaults to 0.

    Returns:
    - tf (float): Final time of the simulation.
    - result (qutip.solver.Result): Result object containing the simulation output.
    """
    # Default values
    if b is None: b = B
    if om_r is None: om_r = Om_r
    if om is None: om = omega
    if p is None: p = phi
    if w_p is None: w_p = W_p
    if exp_ops is None: exp_ops = [
        n1&IdN15, n2&IdN15, n3&IdN15,
        n4&IdN15, n5&IdN15, n6&IdN15,
        n7&IdN15, nc&IdN15,
        sx_gs&IdN15, sy_gs&IdN15, sz_gs&IdN15,
        sx_es&IdN15, sy_es&IdN15, sz_es&IdN15,
        IdNV&nit[0] * nit[0].dag(), IdNV&nit[1] * nit[1].dag() #type: ignore
    ]
    # Arguments for the Hamiltonian
    args = {"w": om, "p": p}
    # Time resolution based on dt
    t_bins = 1000 if dt <= 5 else 5000
    
    tf = ti + dt
    
    # Define Hamiltonian and collapse operators based on mode
    match mode:
        case "Free":
            H = H_dua_hf(b, 0.0, phi_H)
            c_ops = L_dua_hf(0.0, k_index=k_index)
        case "MW":
            H = H_dua_hf(b, om_r, phi_H)
            c_ops = L_dua_hf(0.0, k_index=k_index)
        case "Laser":
            H = H_dua_hf(b, 0.0, phi_H)
            c_ops = L_dua_hf(w_p, k_index=k_index)
        case "Laser-MW":
            H = H_dua_hf(b, om_r, phi_H)
            c_ops = L_dua_hf(w_p, k_index=k_index)
        case _:
            raise ValueError('mode must be one of "Free", "MW", "Laser", or "Laser-MW"')

    # Solve the master equation
    match progress_bar:
        case "OFF":
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                e_ops=exp_ops,
                args=args,
                options={"store_states": True},
            )
        case "ON":
            print(f"{mode} {int(i + 1)} \n ti    |    tf \n {ti:.2f} | {tf:.2f}")
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                e_ops=exp_ops,
                args=args,
                options={"store_states": True, "progress_bar": "tqdm"},
            )
        case _:
            raise ValueError('progress_bar must be "ON" or "OFF"')

    return tf, result

In [ ]:
wrs=np.arange(W_p,25*W_p,W_p)
for wr in tqdm(wrs,desc="Laser Power"):
    photolumin(dynamics_zfs_hf,10, 1.0,i_state=(1 / 3 * (n1 + n2 + n3))&(1/2*IdN15),om=D_gs-mu_e*B[2],e_ops=[(pl)&IdN15],om_r=7*np.pi/t2_gs,w_p_r=wr,progress_bar="OFF")
    photolumin(dynamics_dua_hf,10, 1.0,i_state=(1 / 3 * (n1 + n2 + n3))&(1/2*IdN15),om=D_gs-mu_e*B[2],e_ops=[(pl)&IdN15],om_r=7*np.pi/t2_gs,w_p_r=wr,progress_bar="OFF")

## Simulation on a 2x2 Lattice

## Simulation on a Larger Lattice with Random Orientations